In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn3
import os
import seaborn as sns
from scipy import stats
from scipy.io import loadmat
import pingouin as pg

%matplotlib inline

## __Option A:__ Preprocessing of the data:

#### Including aggregation of data from both measurements and across all sessions, including global min-max-scaling and classification as responders / discriminators

#### Please specify the following variables:
`SAVE`: Whether you want to save the preprocessed data (boolean) <br>
`session_for_classification`: The name of the session you would like to use for the classification (string) <br>
`t1`, `t2`: The timepoints that will be used for making the classifcation (e.g. t1=0 and t2=5) (integer) <br>
`STDDEV_FACTOR`: How large (i.e. how many standard-deviations) does the difference between t1 and t2 have to be, to acceppt responder / discriminator classification? (integer or float)<br>

In [ ]:
SAVE = True
session_for_classification = 'acq2'
t1 = 0
t2 = 5
STDDEV_FACTOR = 0

In [ ]:
l_files = ['EDA_single_1Hz_Normalized1s_-1_15_Dennis.xlsx', 'HR_single_1Hz_Normalized1s_-1_15_Dennis.xlsx']
l_sessions = ['preacq', 'acq1', 'acq2', 'gen1', 'gen2', 'ext']

#l_files = ['HR_single_1Hz_Normalized1s_-1_15_Dennis.xlsx']
#l_sessions = ['preacq']

l_dfs_per_measurement = []

for file in l_files:
    if file.startswith('EDA'):
        measurement = 'EDA'
    elif file.startswith('HR'):
        measurement = 'HR'
        
    df_dict =  pd.read_excel(file, sheet_name = None)
    l_sheets = list(df_dict.keys())
    
    l_dfs_per_session = []
    for session in l_sessions:
        
        # Identify all sheets that contain data of the respective session:
        l_session_sheets = []
        for sheet in l_sheets: 
            if session in sheet:
                l_session_sheets.append(sheet)
        
        l_dfs_CSminus_data = []
        l_dfs_CSplus_data = []
        
        for sheet in l_session_sheets:
            # subject as idx name makes sense after transposing the DF later on
            df_dict[sheet].insert(0, '', list(range(-1, 16)))
            df_dict[sheet].set_index('', drop=True, inplace=True)
            
            if sheet.endswith(('CS-', 'CS- ')):
                l_dfs_CSminus_data.append(df_dict[sheet])
            
            if sheet.endswith(('CS+', 'CS+ ')):
                l_dfs_CSplus_data.append(df_dict[sheet])
        
        # Now concat these data together to obtain a single DF for the entire session:
        df_CSminus_per_session = pd.concat(l_dfs_CSminus_data, axis=1)

        df_CSminus_per_session = df_CSminus_per_session.transpose()
        df_CSminus_per_session = df_CSminus_per_session.reset_index()
        df_CSminus_per_session = df_CSminus_per_session.rename(columns={'index': 'subject_id'})
        df_CSminus_per_session.insert(1, 'stim_count', 'to_be_filled')
        df_CSminus_per_session.insert(1, 'value_type', 'abs_CS-')        
        
        df_CSplus_per_session = pd.concat(l_dfs_CSplus_data, axis=1)
        df_CSplus_per_session = df_CSplus_per_session.transpose()
        df_CSplus_per_session = df_CSplus_per_session.reset_index()
        df_CSplus_per_session = df_CSplus_per_session.rename(columns={'index': 'subject_id'})
        df_CSplus_per_session.insert(1, 'stim_count', 'to_be_filled')
        df_CSplus_per_session.insert(1, 'value_type', 'abs_CS+')

        
        #df_per_session = pd.concat([df_CSminus_per_session, df_CSplus_per_session], axis=0)
        for df_temp in [df_CSminus_per_session, df_CSplus_per_session]:
            l_counts = [elem[elem.index('_') + 1:] for elem in list(df_temp['subject_id'].values)]
            df_temp['stim_count'] = l_counts
            l_subject_ids = [elem[: elem.index('E_')] if 'E_' in elem else elem[: elem.index('_')] for elem in list(df_temp['subject_id'].values)]
            df_temp['subject_id'] = l_subject_ids
            df_temp.insert(1, 'measurement', measurement)
            df_temp.insert(1, 'session', session)
            df_temp.insert(1, 'group', 'to_be_filled')
            l_groups = ['control' if elem.startswith('GES') else 'anxiety' for elem in list(df_temp['subject_id'].values)]
            df_temp['group'] = l_groups
        
        df_per_session = pd.concat([df_CSminus_per_session, df_CSplus_per_session], axis=0)
        df_per_session.reset_index(drop=True, inplace=True)
        l_dfs_per_session.append(df_per_session)

    # Concat the reshaped dataframes from all sessions that contain the "absolute" values 
    # Note: already normalized with Jérémys code to stimulus onset -1.0s
    df_abs = pd.concat(l_dfs_per_session, axis=0)
    df_abs.reset_index(drop = True, inplace = True)
    
    l_dfs_per_measurement.append(df_abs)

df = pd.concat(l_dfs_per_measurement, axis=0)
df.reset_index(drop=True, inplace=True)


l_df_per_gen_session = []

#for gen_session in ['gen1', 'gen2']:
for gen_session in ['gen1']:
    if gen_session == 'gen1':
        l_files = ['Gen1.xlsx', 'Gen2.xlsx', 'Gen3.xlsx', 'Gen4.xlsx']
        char = '.'
    if gen_session == 'gen2':
        l_files = ['Gen1_2.xlsx', 'Gen2_2.xlsx', 'Gen3_2.xlsx', 'Gen4_2.xlsx']
        char = '_'
    
    l_dfs_per_GS = []
    
    for file in l_files:
        stimulus = file[:file.index(char)]
        stimulus = stimulus.replace('en', 'S')
        
        l_dfs_individual_measurements = []
        
        for measurement in ['HR', 'EDA']:
            if measurement == 'HR':
                sheet_name = 'Singles'
            elif measurement == 'EDA':
                sheet_name = 'SinglesSC'
            
            df_temp = pd.read_excel(file, sheet_name = sheet_name)

            df_temp.insert(0, '', list(range(-1, 16)))
            df_temp.set_index('', drop=True, inplace=True)

            df_temp = df_temp.transpose()
            df_temp = df_temp.reset_index()
            df_temp = df_temp.rename(columns={'index': 'subject_id'})
            df_temp.insert(1, 'stim_count', 'to_be_filled')
            df_temp.insert(1, 'value_type', 'abs_' + stimulus)

            l_counts = [elem[elem.index('_') + 1:] for elem in list(df_temp['subject_id'].values)]    
            df_temp['stim_count'] = l_counts 
            l_subject_ids = [elem[: elem.index('E_')] if 'E_' in elem else elem[: elem.index('_')] for elem in list(df_temp['subject_id'].values)]
            df_temp['subject_id'] = l_subject_ids
            # Remember to change to generic!
            df_temp.insert(1, 'measurement', measurement)
            df_temp.insert(1, 'session', gen_session)
            df_temp.insert(1, 'group', 'to_be_filled')
            l_groups = ['control' if elem.startswith('GES') else 'anxiety' for elem in list(df_temp['subject_id'].values)]
            df_temp['group'] = l_groups
            
            l_dfs_individual_measurements.append(df_temp)
            
        df_individual_GS = pd.concat(l_dfs_individual_measurements, axis=0)
        df_individual_GS.reset_index(drop=True, inplace=True)
        
        l_dfs_per_GS.append(df_individual_GS)
    
    df_all_GS_one_session = pd.concat(l_dfs_per_GS, axis=0)
    df_all_GS_one_session.reset_index(drop=True, inplace=True)
    
    l_df_per_gen_session.append(df_all_GS_one_session)
    
df_all_GS_both_sessions = pd.concat(l_df_per_gen_session, axis=0)
df_all_GS_both_sessions.reset_index(drop=True, inplace=True)

df = pd.concat([df, df_all_GS_both_sessions], axis=0)
df.reset_index(drop=True, inplace=True)

# Now global normalization is done
l_dfs_global_norm = []

for measurement in ['EDA', 'HR']:
    
    l_value_types = ['abs_CS+', 'abs_CS-', 'abs_GS1', 'abs_GS2', 'abs_GS3', 'abs_GS4']
    # Select the data
    df_temp = df.loc[(df['measurement'] == measurement) & (df['value_type'].isin(l_value_types))].copy()

    # Since the data contains positive and negative values, the absolute value of the min is added to each value
    # This forces all values to be positive and regular min-max-scaling can be performed
    global_min = df_temp.iloc[:, 6:23].min().min()
    df_temp.iloc[:, 6:23] = df_temp.iloc[:, 6:23] + abs(global_min)
    new_min = df_temp.iloc[:, 6:23].min().min()
    new_max = df_temp.iloc[:, 6:23].max().max()
    df_temp.iloc[:, 6:23] = (df_temp.iloc[:, 6:23] - new_min) / (new_max - new_min)
    for value_type in l_value_types:
        wo_prefix = value_type[value_type.index('_'):]
        df_temp.loc[df_temp['value_type'] == value_type, 'value_type'] = 'norm_global' + wo_prefix
    l_dfs_global_norm.append(df_temp)


df_for_pooling_acq = pd.concat(l_dfs_global_norm, axis = 0)
df_for_pooling_acq = df_for_pooling_acq.loc[df_for_pooling_acq['session'].isin(['acq1', 'acq2'])].copy()

for i in range(1,7):
    old_stim_count = str(i)
    new_stim_count = str(i + 6)
    df_for_pooling_acq.loc[(df_for_pooling_acq['session'] == 'acq2') & (df_for_pooling_acq['stim_count'] == old_stim_count), 'stim_count'] = new_stim_count

df_for_pooling_acq['session'] = 'acq_pooled'


df = pd.concat([df] + l_dfs_global_norm + [df_for_pooling_acq], axis = 0)
df.reset_index(drop=True, inplace=True)

df_to_check1 = df.copy()

df_for_pooling_ext = df.loc[df['session'] == 'ext'].copy()
df_for_pooling_ext['session'] = 'ext_pooled'

df.loc[(df['session'] == 'ext') & (df['stim_count'].isin(['1', '2', '3', '4', '5', '6'])), 'session'] = 'ext1'
df.loc[(df['session'] == 'ext') & (df['stim_count'].isin(['7', '8', '9', '10', '11', '12'])), 'session'] = 'ext2'
df.loc[(df['session'] == 'ext') & (df['stim_count'].isin(['13', '14', '15', '16', '17', '18'])), 'session'] = 'ext3'

df_to_check2 = df.copy()

df = pd.concat([df, df_for_pooling_ext], axis=0)
df.reset_index(drop=True, inplace=True)

df_to_check3 = df.copy()

l_subjects = df['subject_id'].unique()


for measurement in ['EDA', 'HR']:
    for subject in l_subjects:
        for session in ['preacq', 'acq1', 'acq2', 'gen2', 'ext1', 'ext2', 'ext3', 'acq_pooled', 'ext_pooled']:
            
            df_temp = df.loc[(df['measurement'] == measurement) & (df['session'] == session) 
                                     & (df['subject_id'] == subject) & (df['value_type'].isin(['norm_global_CS+', 'norm_global_CS-']))].copy()            
            
            df_meta = df_temp.iloc[:3, 0:6].copy()
            df_meta['value_type'] = ['norm_global_CS+', 'norm_global_CS-', 'norm_global_discrim_ratio']
            df_meta['stim_count'] = ['session_mean', 'session_mean', 'session_mean']
            df_meta.reset_index(drop=True, inplace=True)

            series_CSplus = df_temp.loc[df_temp['value_type'] == 'norm_global_CS+'].iloc[:,6:23].mean()
            series_CSminus = df_temp.loc[df_temp['value_type'] == 'norm_global_CS-'].iloc[:,6:23].mean()
            df_temp_CSplus = series_CSplus.to_frame().T
            df_temp_CSminus = series_CSminus.to_frame().T
            
            df_discrim = pd.concat([df_temp_CSplus, df_temp_CSminus])

            df_discrim = df_discrim.append(df_discrim.iloc[0,:] / (df_discrim.iloc[0,:] + df_discrim.iloc[1,:]), ignore_index=True)

            df_to_append = pd.concat([df_meta, df_discrim], axis=1)

            df = df.append(df_to_append, ignore_index=True)
        
        #for session in ['gen1', 'gen2']:
        for session in ['gen1']:
            df_temp = df.loc[(df['measurement'] == measurement) & (df['session'] == session) 
                         & (df['subject_id'] == subject) & (df['value_type'].isin(['norm_global_CS+', 'norm_global_CS-',
                                                                                   'norm_global_GS1', 'norm_global_GS2',
                                                                                  'norm_global_GS3', 'norm_global_GS4']))].copy()            
            
            df_meta = df_temp.iloc[:11, 0:6].copy()
            df_meta['value_type'] = ['norm_global_CS+', 'norm_global_CS-',
                                     'norm_global_GS1', 'norm_global_GS2', 'norm_global_GS3', 'norm_global_GS4',
                                     'norm_global_discrim_ratio',
                                     'norm_global_discrim_ratio_GS1', 'norm_global_discrim_ratio_GS2','norm_global_discrim_ratio_GS3','norm_global_discrim_ratio_GS4']
            df_meta['stim_count'] = ['session_mean', 'session_mean', 
                                     'session_mean', 'session_mean', 'session_mean', 'session_mean',
                                     'session_mean',
                                     'session_mean', 'session_mean', 'session_mean', 'session_mean']
            df_meta.reset_index(drop=True, inplace=True)

            series_CSplus = df_temp.loc[df_temp['value_type'] == 'norm_global_CS+'].iloc[:,6:23].mean()
            series_CSminus = df_temp.loc[df_temp['value_type'] == 'norm_global_CS-'].iloc[:,6:23].mean()     

            series_GS1 = df_temp.loc[df_temp['value_type'] == 'norm_global_GS1'].iloc[:,6:23].mean()
            series_GS2 = df_temp.loc[df_temp['value_type'] == 'norm_global_GS2'].iloc[:,6:23].mean()
            series_GS3 = df_temp.loc[df_temp['value_type'] == 'norm_global_GS3'].iloc[:,6:23].mean()
            series_GS4 = df_temp.loc[df_temp['value_type'] == 'norm_global_GS4'].iloc[:,6:23].mean()
            
            df_temp_CSplus = series_CSplus.to_frame().T
            df_temp_CSminus = series_CSminus.to_frame().T    
            
            df_temp_GS1 = series_GS1.to_frame().T
            df_temp_GS2 = series_GS2.to_frame().T
            df_temp_GS3 = series_GS3.to_frame().T
            df_temp_GS4 = series_GS4.to_frame().T
            
            df_discrim = pd.concat([df_temp_CSplus, df_temp_CSminus, df_temp_GS1, df_temp_GS2, df_temp_GS3, df_temp_GS4])
            
            df_discrim = df_discrim.append(df_discrim.iloc[0,:] / (df_discrim.iloc[0,:] + df_discrim.iloc[1,:]), ignore_index=True)
            
            df_discrim = df_discrim.append(df_discrim.iloc[0,:] / (df_discrim.iloc[0,:] + df_discrim.iloc[2,:]), ignore_index=True)
            df_discrim = df_discrim.append(df_discrim.iloc[0,:] / (df_discrim.iloc[0,:] + df_discrim.iloc[3,:]), ignore_index=True)
            df_discrim = df_discrim.append(df_discrim.iloc[0,:] / (df_discrim.iloc[0,:] + df_discrim.iloc[4,:]), ignore_index=True)
            df_discrim = df_discrim.append(df_discrim.iloc[0,:] / (df_discrim.iloc[0,:] + df_discrim.iloc[5,:]), ignore_index=True)
            

            #df_discrim.reset_index(drop=True, inplace=True)
            
            df_to_append = pd.concat([df_meta, df_discrim], axis=1)
            
            df = df.append(df_to_append, ignore_index=True)

if SAVE:
    df.to_csv('all_normalized_single_data_with_gen1_and_pooling.csv')


# Classification of each subject as responder and discriminator (each can be different for the individual measurement, e.g. HR responder & EDA non-responder)

# If-condition only added to make multiple subsequent computations possible (column can´t be added if it already exists)
if 'EDA_discriminator' not in df.columns:
    df.insert(2, 'EDA_discriminator', 'to_be_assigned')
    df.insert(2, 'EDA_responder', 'to_be_assigned')
    df.insert(2, 'HR_discriminator', 'to_be_assigned')
    df.insert(2, 'HR_responder', 'to_be_assigned')

# iterate through both measurements individually
for MEASUREMENT in ['HR', 'EDA']:
    for subject in df['subject_id'].unique():
        
        # select all data of the respective subject from the session that is to be used for the classification as series, since there are multiple presentations of each stimulus
        # CS- at t1
        series_CSminus_t1 = df.loc[(df['session'] == session_for_classification) & (df['measurement'] == MEASUREMENT) 
                                    & (df['value_type'] == 'norm_global_CS-') & (df['stim_count'] != 'session_mean') & (df['subject_id'] == subject), t1]
        # CS- at t2
        series_CSminus_t2 =  df.loc[(df['session'] == session_for_classification) & (df['measurement'] == MEASUREMENT) 
                                    & (df['value_type'] == 'norm_global_CS-') & (df['stim_count'] != 'session_mean') & (df['subject_id'] == subject), t2]
        # CS+ at t1
        series_CSplus_t1 = df.loc[(df['session'] == session_for_classification) & (df['measurement'] == MEASUREMENT) 
                                    & (df['value_type'] == 'norm_global_CS+') & (df['stim_count'] != 'session_mean') & (df['subject_id'] == subject), t1]
        # CS+ at t2
        series_CSplus_t2 =  df.loc[(df['session'] == session_for_classification) & (df['measurement'] == MEASUREMENT) 
                                    & (df['value_type'] == 'norm_global_CS+') & (df['stim_count'] != 'session_mean') & (df['subject_id'] == subject), t2]        
        
        # Now compute a series that contains the differences for each pair of presentations:
        series_diff_CSminus = series_CSminus_t2 - series_CSminus_t1
        series_diff_CSplus = series_CSplus_t2 - series_CSplus_t1  

        # Since the definitions of responder and discriminator are different for the individual measurements, this section is split
        if MEASUREMENT == 'HR':
            # First, let´s check whether the individual fullfills the criteria of being a HR responder, i.e.:
            # The difference in the HR between t2 and t1 is negative (= a decrease in HR)
            # The difference between t2 and t1 in the mean HR responses after the CS+ is larger than STDDEV_FACTOR * the standard deviation of the responses at t1 
            if (series_diff_CSplus.mean() < 0) & (abs(series_diff_CSplus.mean()) > STDDEV_FACTOR*series_CSplus_t1.std()):
                responder = True
            else:
                responder = False
            # Now let´s check whether the individual fullfills the criteria of being a HR discriminator, i.e.:
            # The subject is already classified as being a responder
            # The difference in the HR differences (between t1 and t2) after presentation of the CS+ or CS-, is larger than STDDEV_FACTOR* the standard deviation of the CS- differences
            if (responder == True) & (series_diff_CSplus.mean() < (series_diff_CSminus.mean() - STDDEV_FACTOR*series_diff_CSminus.std())):
                discriminator = True
            else:
                discriminator = False
            
            
        elif MEASUREMENT == 'EDA':
            # Similar as for HR, but now we check for an increase in EDA between t1 and t2:
            if (series_diff_CSplus.mean() > 0) & (abs(series_diff_CSplus.mean()) > STDDEV_FACTOR*series_CSplus_t1.std()):
                responder = True
            else:
                responder = False
            # Similar as for HR, but now we check for a larger increase in EDA between t1 and t2 after CS+ presentations, compared to after CS- presentations
            if (responder == True) & (series_diff_CSplus.mean() > (series_diff_CSminus.mean() + STDDEV_FACTOR*series_diff_CSminus.std())):
                discriminator = True
            else:
                discriminator = False
        
        # The results are stored in the DataFrame
        df.loc[df['subject_id'] == subject, MEASUREMENT + '_responder'] = responder
        df.loc[df['subject_id'] == subject, MEASUREMENT + '_discriminator'] = discriminator

        
        
# Append baseline measures: 

df_baseline_anx = pd.read_excel('HR+RMSSD.xls', sheet_name = 'ANXKJP')
df_baseline_ges = pd.read_excel('HR+RMSSD.xls', sheet_name = 'GESKJP')


df['baseline_HR_Phase1'] = np.NaN
df['baseline_RMSSD_Phase1'] = np.NaN

for subject in df['subject_id'].unique():
    if subject.startswith('ANX'):
        # Get baseline values:
        baseline_HR = df_baseline_anx.loc[df_baseline_anx['Subject'] == subject, 'HR_Phase1'].values[0]
        baseline_RMSSD = df_baseline_anx.loc[df_baseline_anx['Subject'] == subject, 'RMSSD_Phase1'].values[0]
        
    elif subject.startswith('GES'):
        # Get baseline values:
        baseline_HR = df_baseline_ges.loc[df_baseline_ges['Subject'] == subject, 'HR_Phase1'].values[0]
        baseline_RMSSD = df_baseline_ges.loc[df_baseline_ges['Subject'] == subject, 'RMSSD_Phase1'].values[0]       
        
    # Copy them to the main DataFrame:
    df.loc[df['subject_id'] == subject, 'baseline_HR_Phase1'] = baseline_HR
    df.loc[df['subject_id'] == subject, 'baseline_RMSSD_Phase1'] = baseline_RMSSD
        
if SAVE:
    df.to_csv('all_normalized_and_classified_single_data_with_gen1_and_pooling.csv') 

df.head()

## Option B: Read the already preprocessed data:

This actually throws an error for certain computations, due to an error of the dtypes (probably objects instead of int, should be easy to fix)

df = pd.read_csv('all_normalized_and_classified_single_data_with_gen1.csv', index_col = 0)
df.head()

# Let´s start the analyses with the classical "controls vs. anxious" part:

#### First of, here is an overview of the mean responses from all individual subjects and the mean group response

In [ ]:
VALUE_TYPE = 'norm_global_CS-'
STIM_COUNT = 'session_mean'
MEASUREMENT = 'HR'
SESSION = 'ext_pooled'

#### Since this is the first plot, the following cell will setup some features of the figure layout and can be used to change the layout & design of all subsequent figures

In [ ]:
TITLE_SIZE = 17
SUBTITLE_SIZE = 15
AXES_LABEL_SIZE = 14
COLOR_CTRL = 'g'
COLOR_ANX = 'm'

#### Since each subject receives multiple stimulus presentations per session, we can also plot the response to each presentation individually & compare them to the session mean

In [ ]:
#SUBJECT_ID = df['subject_id'].unique()[0]
SUBJECT_ID = 'GESKJP016'

In [ ]:
# Depending on whether df is created by reading a .csv file or from preprocessing, the timepoints seem to be stored differently (as int or as str). To catch this error:
if '-1' in df.columns:
    column_a, column_b = '-1', '15'
elif -1 in df.columns:
    column_a, column_b = -1, 15

# To make this plot highly flexible, e.g. if additional columns are added with regard to responder classifications and so on, the column indices are not hard coded:
idx_start = list(df.columns).index(column_a)
idx_stop = list(df.columns).index(column_b)

df_CSminus = df.loc[(df['session'] == SESSION) & (df['value_type'] == 'norm_global_CS-') & (df['measurement'] == MEASUREMENT) 
                & (df['stim_count'] != 'session_mean') & (df['subject_id'] == SUBJECT_ID)].copy()
df_CSplus = df.loc[(df['session'] == SESSION) & (df['value_type'] == 'norm_global_CS+') & (df['measurement'] == MEASUREMENT) 
                & (df['stim_count'] != 'session_mean') & (df['subject_id'] == SUBJECT_ID)].copy()

mean_CSminus = df_CSminus.iloc[:, idx_start:idx_stop].mean()
sem_CSminus = df_CSminus.iloc[:, idx_start:idx_stop].sem()
mean_CSplus = df_CSplus.iloc[:, idx_start:idx_stop].mean()
sem_CSplus = df_CSplus.iloc[:, idx_start:idx_stop].sem()

group_id = df.loc[df['subject_id'] == SUBJECT_ID, 'group'].iloc[0]

if group_id == 'control':
    color = COLOR_CTRL
elif group_id == 'anxiety':
    color = COLOR_ANX

fig = plt.figure(figsize=(18, 8))
gs = fig.add_gridspec(1, 2)

ax1 = fig.add_subplot(gs[0, 0])
df_CSminus.iloc[:, idx_start:idx_stop].transpose().plot(legend=False, alpha=.2, color=color, ax=ax1)
plt.errorbar(x = list(mean_CSminus.index), y = mean_CSminus.values, yerr = sem_CSminus.values, color=color)
plt.title('Responses to CS-', fontsize = SUBTITLE_SIZE)
plt.xlabel('time in [s] from stimulus onset', fontsize = AXES_LABEL_SIZE)
plt.ylabel('norm global CS-', fontsize = AXES_LABEL_SIZE)
#plt.ylim(0.25, 0.4)

ax2 = fig.add_subplot(gs[0, 1], sharey=ax1)
df_CSplus.iloc[:, idx_start:idx_stop].transpose().plot(legend=False, alpha=.2, color=color, ax=ax2)
plt.errorbar(x = list(mean_CSplus.index), y = mean_CSplus.values, yerr = sem_CSplus.values, color=color)
plt.title('Responses to CS+', fontsize = SUBTITLE_SIZE)
plt.xlabel('time in [s] from stimulus onset', fontsize = AXES_LABEL_SIZE)
plt.ylabel('norm global CS+', fontsize = AXES_LABEL_SIZE)

plt.suptitle(MEASUREMENT + ' responses of ' + SUBJECT_ID + ' in session: ' + SESSION, fontsize = TITLE_SIZE)
plt.tight_layout()
plt.show()

In [ ]:
# Depending on whether df is created by reading a .csv file or from preprocessing, the timepoints seem to be stored differently (as int or as str). To catch this error:
if '-1' in df.columns:
    column_a, column_b = '-1', '15'
elif -1 in df.columns:
    column_a, column_b = -1, 15

# To make this plot highly flexible, e.g. if additional columns are added with regard to responder classifications and so on, the column indices are not hard coded:
idx_start = list(df.columns).index(column_a)
idx_stop = list(df.columns).index(column_b)


df_anx = df.loc[(df['session'] == SESSION) & (df['value_type'] == VALUE_TYPE) & (df['measurement'] == MEASUREMENT) 
                & (df['stim_count'] == STIM_COUNT) & (df['group'] == 'anxiety')].copy()
df_ctrl = df.loc[(df['session'] == SESSION) & (df['value_type'] == VALUE_TYPE) & (df['measurement'] == MEASUREMENT) 
                & (df['stim_count'] == STIM_COUNT) & (df['group'] == 'control')].copy()

mean_anx = df_anx.iloc[:, idx_start:idx_stop].mean()
sem_anx = df_anx.iloc[:, idx_start:idx_stop].sem()
mean_ctrl = df_ctrl.iloc[:, idx_start:idx_stop].mean()
sem_ctrl = df_ctrl.iloc[:, idx_start:idx_stop].sem()


fig = plt.figure(figsize=(18, 8))
gs = fig.add_gridspec(1, 2)

ax1 = fig.add_subplot(gs[0, 0])
df_ctrl.iloc[:, idx_start:idx_stop].transpose().plot(legend=False, alpha=.2, color=COLOR_CTRL, ax=ax1)
plt.errorbar(x = list(mean_ctrl.index), y = mean_ctrl.values, yerr = sem_ctrl.values, color=COLOR_CTRL)
plt.title('Control group', fontsize=SUBTITLE_SIZE)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel(VALUE_TYPE, fontsize=AXES_LABEL_SIZE)
#plt.ylim(0.25, 0.4)

ax2 = fig.add_subplot(gs[0, 1], sharey=ax1)
df_anx.iloc[:, idx_start:idx_stop].transpose().plot(legend=False, alpha=.2, color=COLOR_ANX, ax=ax2)
plt.errorbar(x = list(mean_anx.index), y = mean_anx.values, yerr = sem_anx.values, color=COLOR_ANX)
plt.title('Anxiety group', fontsize=SUBTITLE_SIZE)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel(VALUE_TYPE, fontsize=AXES_LABEL_SIZE)

plt.suptitle(VALUE_TYPE + ' for ' + MEASUREMENT + ' in session: ' + SESSION, fontsize = TITLE_SIZE)
plt.tight_layout()
plt.show()

## Next, let´s inspect the discrimination ratio and the development of responses of the course of the experiment

#### For this, we plot all responses for a given measurement and the resulting discrimination ratios across all sessions to get a first overview

In [ ]:
MEASUREMENT = 'HR'

In [ ]:
# Depending on whether df is created by reading a .csv file or from preprocessing, the timepoints seem to be stored differently (as int or as str). To catch this error:
if '-1' in df.columns:
    column_a, column_b = '-1', '15'
elif -1 in df.columns:
    column_a, column_b = -1, 15

# To make this plot highly flexible, e.g. if additional columns are added with regard to responder classifications and so on, the column indices are not hard coded:
idx_start = list(df.columns).index(column_a)
idx_stop = list(df.columns).index(column_b)


# Create figure
fig = plt.figure(figsize=(20, 40), facecolor='white')
gs = fig.add_gridspec(8, 2)

l_sessions = ['preacq', 'acq1', 'acq2', 'gen1', 'gen2', 'ext1', 'ext2', 'ext3']


SESSION = 'preacq'

# Select data
df_ctrl = df.loc[(df['session'] == SESSION) & (df['group'] == 'control') & (df['measurement'] == MEASUREMENT)].copy()
df_anx = df.loc[(df['session'] == SESSION) & (df['group'] == 'anxiety') & (df['measurement'] == MEASUREMENT)].copy()

anx_norm_cs_p = df_anx.loc[df_anx['value_type'] == 'norm_global_CS+'].iloc[:, idx_start:idx_stop].mean()
anx_norm_cs_m = df_anx.loc[df_anx['value_type'] == 'norm_global_CS-'].iloc[:, idx_start:idx_stop].mean()
anx_discrim = df_anx.loc[df_anx['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
anx_discrim_sem = df_anx.loc[df_anx['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()

ctrl_norm_cs_p = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_CS+'].iloc[:, idx_start:idx_stop].mean()
ctrl_norm_cs_m = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_CS-'].iloc[:, idx_start:idx_stop].mean()
ctrl_discrim = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
ctrl_disrcim_sem = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()


# Create the plots:
ax1 = fig.add_subplot(gs[0, 0])
plt.plot(anx_norm_cs_p, color = COLOR_ANX, alpha = 1, label='anxiety CS+')
plt.plot(anx_norm_cs_m, color = COLOR_ANX, alpha = 1, linestyle = 'dashed', label='anxiety CS-')

plt.plot(ctrl_norm_cs_p, color = COLOR_CTRL, alpha = 1, label='control CS+')
plt.plot(ctrl_norm_cs_m, color = COLOR_CTRL, alpha = 1, linestyle = 'dashed', label='control CS-')
#plt.errorbar(x = list(anx_discrim.index), y = anx_discrim.values, yerr = anx_discrim_sem.values, color='m', label='discrim. ratio')
#plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel('normalized CS responses', fontsize=AXES_LABEL_SIZE)
plt.title('CS responses during: {}'.format(SESSION), fontsize=SUBTITLE_SIZE)
plt.legend(loc = 'lower left')

#fig.add_subplot(gs[0,1], sharey=f_ax1)

#plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color='g', label='discrim. ratio')
#plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
#plt.title('Control group')
#plt.xlabel('time in [s] from stimulus onset')
#plt.ylabel('normalized CS responses / discrimination ratio')
#plt.legend(loc = 'lower left')

ax2 = fig.add_subplot(gs[0,1])
#fig.add_subplot(gs[0,2], sharey=f_ax1)
plt.errorbar(x = list(anx_discrim.index), y = anx_discrim.values, yerr = anx_discrim_sem.values, color=COLOR_ANX, label='anxiety')
plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color=COLOR_CTRL, label='control')
plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
plt.title('Discrimination ratios during: {}'.format(SESSION), fontsize=SUBTITLE_SIZE)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel('discrimination ratio', fontsize=AXES_LABEL_SIZE)
plt.legend(loc = 'lower left')


for column in range(1,8):
    SESSION = l_sessions[column]
    
    # Select data
    df_ctrl = df.loc[(df['session'] == SESSION) & (df['group'] == 'control') & (df['measurement'] == MEASUREMENT)].copy()
    df_anx = df.loc[(df['session'] == SESSION) & (df['group'] == 'anxiety') & (df['measurement'] == MEASUREMENT)].copy()

    anx_norm_cs_p = df_anx.loc[df_anx['value_type'] == 'norm_global_CS+'].iloc[:, idx_start:idx_stop].mean()
    anx_norm_cs_m = df_anx.loc[df_anx['value_type'] == 'norm_global_CS-'].iloc[:, idx_start:idx_stop].mean()
    anx_discrim = df_anx.loc[df_anx['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
    anx_discrim_sem = df_anx.loc[df_anx['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()

    ctrl_norm_cs_p = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_CS+'].iloc[:, idx_start:idx_stop].mean()
    ctrl_norm_cs_m = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_CS-'].iloc[:, idx_start:idx_stop].mean()
    ctrl_discrim = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
    ctrl_disrcim_sem = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()
    

    # Create the plots:
    fig.add_subplot(gs[column, 0], sharey=ax1)
    plt.plot(anx_norm_cs_p, color = COLOR_ANX, alpha = 1, label='anxiety CS+')
    plt.plot(anx_norm_cs_m, color = COLOR_ANX, alpha = 1, linestyle = 'dashed', label='anxiety CS-')

    plt.plot(ctrl_norm_cs_p, color = COLOR_CTRL, alpha = 1, label='control CS+')
    plt.plot(ctrl_norm_cs_m, color = COLOR_CTRL, alpha = 1, linestyle = 'dashed', label='control CS-')
    #plt.errorbar(x = list(anx_discrim.index), y = anx_discrim.values, yerr = anx_discrim_sem.values, color='m', label='discrim. ratio')
    #plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
    plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
    plt.ylabel('normalized CS responses', fontsize=AXES_LABEL_SIZE)
    plt.title('CS responses during: {}'.format(SESSION), fontsize=SUBTITLE_SIZE)
    plt.legend(loc = 'lower left')

    #fig.add_subplot(gs[0,1], sharey=f_ax1)

    #plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color='g', label='discrim. ratio')
    #plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
    #plt.title('Control group')
    #plt.xlabel('time in [s] from stimulus onset')
    #plt.ylabel('normalized CS responses / discrimination ratio')
    #plt.legend(loc = 'lower left')

    fig.add_subplot(gs[column,1], sharey=ax2)
    #fig.add_subplot(gs[0,2], sharey=f_ax1)
    plt.errorbar(x = list(anx_discrim.index), y = anx_discrim.values, yerr = anx_discrim_sem.values, color=COLOR_ANX, label='anxiety')
    plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color=COLOR_CTRL, label='control')
    plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
    plt.title('Discrimination ratios during: {}'.format(SESSION), fontsize=SUBTITLE_SIZE)
    plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
    plt.ylabel('discrimination ratio', fontsize=AXES_LABEL_SIZE)
    plt.legend(loc = 'lower left')

plt.tight_layout()

#plt.savefig('Development_of_HR_responses_and_discrimination_ratio_across_all_sessions.png', dpi=300)
plt.show()

## Now let´s do the statistics:

#### For this, we plot the group mean responses for a given session & measurement and compare the responses to the two CS and the discrimination ratios

`SESSION`: 'preacq', 'acq1', 'acq2', 'gen1', 'gen2', 'ext1', 'ext2', 'ext3' <br>
`MEASUREMENT`: 'HR', 'EDA'<br>

#### The results of statistical comparisons will be displayed below the plot. A mixed-model-ANOVA (MMA) and the pairwise comparisons for each timepoint will be calculated.

In [ ]:
SESSION = 'acq_pooled' 
MEASUREMENT = 'HR'

In [ ]:
# Depending on whether df is created by reading a .csv file or from preprocessing, the timepoints seem to be stored differently (as int or as str). To catch this error:
if '-1' in df.columns:
    column_a, column_b = '-1', '15'
elif -1 in df.columns:
    column_a, column_b = -1, 15

# To make this plot highly flexible, e.g. if additional columns are added with regard to responder classifications and so on, the column indices are not hard coded:
idx_start = list(df.columns).index(column_a)
idx_stop = list(df.columns).index(column_b)


# Select data
df_ctrl = df.loc[(df['session'] == SESSION) & (df['group'] == 'control') & (df['measurement'] == MEASUREMENT)].copy()
df_anx = df.loc[(df['session'] == SESSION) & (df['group'] == 'anxiety') & (df['measurement'] == MEASUREMENT)].copy()

anx_norm_cs_p = df_anx.loc[df_anx['value_type'] == 'norm_global_CS+'].iloc[:, idx_start:idx_stop].mean()
anx_norm_cs_m = df_anx.loc[df_anx['value_type'] == 'norm_global_CS-'].iloc[:, idx_start:idx_stop].mean()
anx_discrim = df_anx.loc[df_anx['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
anx_discrim_sem = df_anx.loc[df_anx['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()

ctrl_norm_cs_p = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_CS+'].iloc[:, idx_start:idx_stop].mean()
ctrl_norm_cs_m = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_CS-'].iloc[:, idx_start:idx_stop].mean()
ctrl_discrim = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
ctrl_disrcim_sem = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()


# Create figure
fig = plt.figure(figsize=(18, 8))
gs = fig.add_gridspec(1, 2)

f_ax1 = fig.add_subplot(gs[0, 0])
plt.plot(anx_norm_cs_p, color = COLOR_ANX, alpha = 1, label='anxiety CS+')
plt.plot(anx_norm_cs_m, color = COLOR_ANX, alpha = 1, linestyle = 'dashed', label='anxiety CS-')

plt.plot(ctrl_norm_cs_p, color = COLOR_CTRL, alpha = 1, label='control CS+')
plt.plot(ctrl_norm_cs_m, color = COLOR_CTRL, alpha = 1, linestyle = 'dashed', label='control CS-')
#plt.errorbar(x = list(anx_discrim.index), y = anx_discrim.values, yerr = anx_discrim_sem.values, color='m', label='discrim. ratio')
#plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel('normalized CS responses', fontsize=AXES_LABEL_SIZE)
plt.title('CS responses', fontsize=SUBTITLE_SIZE)
plt.legend(loc = 'lower left')

#fig.add_subplot(gs[0,1], sharey=f_ax1)

#plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color='g', label='discrim. ratio')
#plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
#plt.title('Control group')
#plt.xlabel('time in [s] from stimulus onset')
#plt.ylabel('normalized CS responses / discrimination ratio')
#plt.legend(loc = 'lower left')

fig.add_subplot(gs[0,1])
#fig.add_subplot(gs[0,2], sharey=f_ax1)
plt.errorbar(x = list(anx_discrim.index), y = anx_discrim.values, yerr = anx_discrim_sem.values, color=COLOR_ANX, label='anxiety')
plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color=COLOR_CTRL, label='control')
plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
plt.title('Discrimination ratios', fontsize=SUBTITLE_SIZE)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel('discrimination ratio', fontsize=AXES_LABEL_SIZE)
plt.legend(loc = 'lower left')

plt.suptitle(MEASUREMENT + ' in session: ' + SESSION, fontsize = TITLE_SIZE)
plt.tight_layout()

plt.show()


# Now also compute the corresponding statistics:

VALUE_TYPE = 'discrim_ratio'

# Depending on whether df is created by reading a .csv file or from preprocessing, the timepoints seem to be stored differently (as int or as str). To catch this error:
if '-1' in df.columns:
    l_timepoints = [str(elem) for elem in list(range(0, 16))]
elif -1 in df.columns:
    l_timepoints = list(range(0, 16))

l_dfs_per_timepoint = []

for timepoint in l_timepoints:
    df_temp = df.loc[(df['session'] == SESSION) & (df['value_type'] == 'norm_global_' + VALUE_TYPE) & (df['measurement'] == MEASUREMENT)
                     & (df['stim_count'] == 'session_mean'), ['subject_id', 'group', 'measurement', 'session', 'value_type', timepoint]].copy()
    l_headers = list(df_temp.columns)
    l_headers[-1] = 'data'
    df_temp.columns = l_headers
    df_temp['timepoint'] = float(timepoint)
    l_dfs_per_timepoint.append(df_temp)
    
df_stats = pd.concat(l_dfs_per_timepoint)

results_mixed_anova = pg.mixed_anova(data=df_stats, dv='data', between='group', within='timepoint', subject='subject_id')
p_groups, p_timepoints, p_interactions = results_mixed_anova.loc[0,'p-unc'], results_mixed_anova.loc[1,'p-unc'], results_mixed_anova.loc[2,'p-unc']

df_pairwise = pg.pairwise_ttests(data=df_stats, dv='data', between='group', within='timepoint', subject='subject_id', padjust='bonf')
l_pairwise_p_values = list(df_pairwise.loc[df_pairwise['Contrast'] == 'timepoint * group'].sort_values(by='timepoint').loc[:,'p-unc'].values)

p_values_vs_05 = {}
for group in ['control', 'anxiety']:
    l_p_values_05 = []
    for timepoint in range(0, 16):
        data = df_stats.loc[(df_stats['group'] == group) & (df_stats['timepoint'] == timepoint), 'data'].values
        l_p_values_05.append(round(pg.ttest(x=data, y=0.5).loc['T-test', 'p-val'], 3))
    p_values_vs_05[group] = l_p_values_05

level1 = ['Mixed-Model-ANOVA', 'Mixed-Model-ANOVA', 'Mixed-Model-ANOVA']
level2 = ['Groups', 'Timepoints', 'Interactions']

for x in range(16):
    level1.append('pairwise comparisons per timepoint')
    level2.append(str(x))
    
tuples = list(zip(*[level1, level2]))
index = pd.MultiIndex.from_tuples(tuples)
    
df_stats_summary = pd.DataFrame(index=['p-values Ctrl vs. Anx', 'p-values Ctrl vs. 0.5', 'p-values Anx vs. 0.5'], columns=index)
l_p_values = [p_groups, p_timepoints, p_interactions] + l_pairwise_p_values
l_p_values = [round(elem, 3) for elem in l_p_values]

df_stats_summary.loc['p-values Ctrl vs. Anx', :] = l_p_values
df_stats_summary.iloc[1, 3:] = p_values_vs_05['control']
df_stats_summary.iloc[2, 3:] = p_values_vs_05['anxiety']

df_stats_summary

## We can also take a closer look at the responses to the morphed faces during the generalization sessions:

In [ ]:
GEN_SESSION = 'gen1'
MEASUREMENT = 'EDA'

In [ ]:
# Depending on whether df is created by reading a .csv file or from preprocessing, the timepoints seem to be stored differently (as int or as str). To catch this error:
if '-1' in df.columns:
    column_a, column_b = '-1', '15'
elif -1 in df.columns:
    column_a, column_b = -1, 15

# To make this plot highly flexible, e.g. if additional columns are added with regard to responder classifications and so on, the column indices are not hard coded:
idx_start = list(df.columns).index(column_a)
idx_stop = list(df.columns).index(column_b)


# Select data
df_ctrl = df.loc[(df['session'] == GEN_SESSION) & (df['group'] == 'control') & (df['measurement'] == MEASUREMENT)].copy()
df_anx = df.loc[(df['session'] == GEN_SESSION) & (df['group'] == 'anxiety') & (df['measurement'] == MEASUREMENT)].copy()


anx_discrim = df_anx.loc[df_anx['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
anx_discrim_sem = df_anx.loc[df_anx['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()

anx_discrim_GS1 = df_anx.loc[df_anx['value_type'] == 'norm_global_discrim_ratio_GS1'].iloc[:, idx_start:idx_stop].mean()
anx_discrim_GS1_sem = df_anx.loc[df_anx['value_type'] == 'norm_global_discrim_ratio_GS1'].iloc[:, idx_start:idx_stop].sem()

anx_discrim_GS2 = df_anx.loc[df_anx['value_type'] == 'norm_global_discrim_ratio_GS2'].iloc[:, idx_start:idx_stop].mean()
anx_discrim_GS2_sem = df_anx.loc[df_anx['value_type'] == 'norm_global_discrim_ratio_GS2'].iloc[:, idx_start:idx_stop].sem()

anx_discrim_GS3 = df_anx.loc[df_anx['value_type'] == 'norm_global_discrim_ratio_GS3'].iloc[:, idx_start:idx_stop].mean()
anx_discrim_GS3_sem = df_anx.loc[df_anx['value_type'] == 'norm_global_discrim_ratio_GS3'].iloc[:, idx_start:idx_stop].sem()

anx_discrim_GS4 = df_anx.loc[df_anx['value_type'] == 'norm_global_discrim_ratio_GS4'].iloc[:, idx_start:idx_stop].mean()
anx_discrim_GS4_sem = df_anx.loc[df_anx['value_type'] == 'norm_global_discrim_ratio_GS4'].iloc[:, idx_start:idx_stop].sem()


ctrl_discrim = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
ctrl_discrim_sem = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()

ctrl_discrim_GS1 = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_discrim_ratio_GS1'].iloc[:, idx_start:idx_stop].mean()
ctrl_discrim_GS1_sem = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_discrim_ratio_GS1'].iloc[:, idx_start:idx_stop].sem()

ctrl_discrim_GS2 = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_discrim_ratio_GS2'].iloc[:, idx_start:idx_stop].mean()
ctrl_discrim_GS2_sem = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_discrim_ratio_GS2'].iloc[:, idx_start:idx_stop].sem()

ctrl_discrim_GS3 = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_discrim_ratio_GS3'].iloc[:, idx_start:idx_stop].mean()
ctrl_discrim_GS3_sem = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_discrim_ratio_GS3'].iloc[:, idx_start:idx_stop].sem()

ctrl_discrim_GS4 = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_discrim_ratio_GS4'].iloc[:, idx_start:idx_stop].mean()
ctrl_discrim_GS4_sem = df_ctrl.loc[df_ctrl['value_type'] == 'norm_global_discrim_ratio_GS4'].iloc[:, idx_start:idx_stop].sem()

# Create figure
fig = plt.figure(figsize=(18, 8), facecolor='white')
gs = fig.add_gridspec(1, 2)

f_ax1 = fig.add_subplot(gs[0, 0])
plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_discrim_sem.values, color=COLOR_CTRL, label='CS-')
plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim_GS1.values, yerr = ctrl_discrim_GS1_sem.values, color=COLOR_CTRL, label='GS1', alpha=0.8)
plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim_GS2.values, yerr = ctrl_discrim_GS2_sem.values, color=COLOR_CTRL, label='GS2', alpha=0.6)
plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim_GS3.values, yerr = ctrl_discrim_GS3_sem.values, color=COLOR_CTRL, label='GS3', alpha=0.4)
plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim_GS4.values, yerr = ctrl_discrim_GS4_sem.values, color=COLOR_CTRL, label='GS4', alpha=0.2)

#plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color=COLOR_CTRL, label='control')
plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
plt.title('Discrimination ratios - control group', fontsize=SUBTITLE_SIZE)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel('discrimination ratio', fontsize=AXES_LABEL_SIZE)
plt.legend(loc = 'lower left')

#fig.add_subplot(gs[0,1], sharey=f_ax1)

#plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color='g', label='discrim. ratio')
#plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
#plt.title('Control group')
#plt.xlabel('time in [s] from stimulus onset')
#plt.ylabel('normalized CS responses / discrimination ratio')
#plt.legend(loc = 'lower left')

fig.add_subplot(gs[0,1], sharey=f_ax1)
#fig.add_subplot(gs[0,2], sharey=f_ax1)
plt.errorbar(x = list(anx_discrim.index), y = anx_discrim.values, yerr = anx_discrim_sem.values, color=COLOR_ANX, label='CS-')
plt.errorbar(x = list(anx_discrim.index), y = anx_discrim_GS1.values, yerr = anx_discrim_GS1_sem.values, color=COLOR_ANX, label='GS1', alpha=0.8)
plt.errorbar(x = list(anx_discrim.index), y = anx_discrim_GS2.values, yerr = anx_discrim_GS2_sem.values, color=COLOR_ANX, label='GS2', alpha=0.6)
plt.errorbar(x = list(anx_discrim.index), y = anx_discrim_GS3.values, yerr = anx_discrim_GS3_sem.values, color=COLOR_ANX, label='GS3', alpha=0.4)
plt.errorbar(x = list(anx_discrim.index), y = anx_discrim_GS4.values, yerr = anx_discrim_GS4_sem.values, color=COLOR_ANX, label='GS4', alpha=0.2)

#plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color=COLOR_CTRL, label='control')
plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
plt.title('Discrimination ratios - anxiety group', fontsize=SUBTITLE_SIZE)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel('discrimination ratio', fontsize=AXES_LABEL_SIZE)
plt.legend(loc = 'lower left')

plt.suptitle(MEASUREMENT + ' in session: ' + GEN_SESSION, fontsize = TITLE_SIZE)
plt.tight_layout()

#plt.savefig('EDA_Discrimination_ratios_to_different_stimuli_during_gen1.png', dpi=300)
plt.show()

In [ ]:

timepoint = 10 # 'mean_all_timepoints'#'mean_all_timepoints' # 6, 10

l_value_types = ['norm_global_CS+', 'norm_global_GS1', 'norm_global_GS2', 'norm_global_GS3', 'norm_global_GS4', 'norm_global_CS-']
#l_value_types = ['norm_global_discrim_ratio_GS1', 'norm_global_discrim_ratio_GS2', 'norm_global_discrim_ratio_GS3', 'norm_global_discrim_ratio_GS4', 'norm_global_discrim_ratio']

In [ ]:
# A little more preprocessing to plot the data from the generalization sessions as a curve CS+ > GS1 > GS2 > GS3 > GS4 > CS-
if 'mean_all_timepoints' not in df.columns:
    df['mean_all_timepoints'] = df[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]].mean(axis=1)



fig = plt.figure(figsize=(15,15), facecolor='white')
gs = fig.add_gridspec(2, 1)


fig.add_subplot(gs[0,0])
sns.pointplot(data=df.loc[(df['session'] == 'gen1') & (df['measurement'] == 'HR') & (df['stim_count'] == 'session_mean') 
                          & (df['value_type'].isin(l_value_types))], 
              x='value_type', 
              y=timepoint,
              hue='group',
              dodge=True, 
              order=l_value_types,
              palette=[COLOR_CTRL, COLOR_ANX])
plt.title('Heart rates')


fig.add_subplot(gs[1,0])
sns.pointplot(data=df.loc[(df['session'] == 'gen1') & (df['measurement'] == 'EDA') & (df['stim_count'] == 'session_mean') 
                          & (df['value_type'].isin(l_value_types))], 
              x='value_type', 
              y=timepoint,
              hue='group',
              dodge=True, 
              order=l_value_types,
              palette=[COLOR_CTRL, COLOR_ANX])
plt.title('Skin conductance')


plt.show()

## Likewise, we can check how the responses develop in the extinction sessions

In [ ]:
MEASUREMENT = 'HR'

In [ ]:
# Depending on whether df is created by reading a .csv file or from preprocessing, the timepoints seem to be stored differently (as int or as str). To catch this error:
if '-1' in df.columns:
    column_a, column_b = '-1', '15'
elif -1 in df.columns:
    column_a, column_b = -1, 15

# To make this plot highly flexible, e.g. if additional columns are added with regard to responder classifications and so on, the column indices are not hard coded:
idx_start = list(df.columns).index(column_a)
idx_stop = list(df.columns).index(column_b)


# Select data
df_ctrl_1 = df.loc[(df['session'] == 'ext1') & (df['group'] == 'control') & (df['measurement'] == MEASUREMENT)].copy()
df_anx_1 = df.loc[(df['session'] == 'ext1') & (df['group'] == 'anxiety') & (df['measurement'] == MEASUREMENT)].copy()

df_ctrl_2 = df.loc[(df['session'] == 'ext2') & (df['group'] == 'control') & (df['measurement'] == MEASUREMENT)].copy()
df_anx_2 = df.loc[(df['session'] == 'ext2') & (df['group'] == 'anxiety') & (df['measurement'] == MEASUREMENT)].copy()

df_ctrl_3 = df.loc[(df['session'] == 'ext3') & (df['group'] == 'control') & (df['measurement'] == MEASUREMENT)].copy()
df_anx_3 = df.loc[(df['session'] == 'ext3') & (df['group'] == 'anxiety') & (df['measurement'] == MEASUREMENT)].copy()


ctrl_discrim_1 = df_ctrl_1.loc[df_ctrl_1['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
ctrl_discrim_sem_1 = df_ctrl_1.loc[df_ctrl_1['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()

ctrl_discrim_2 = df_ctrl_2.loc[df_ctrl_2['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
ctrl_discrim_sem_2 = df_ctrl_2.loc[df_ctrl_2['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()

ctrl_discrim_3 = df_ctrl_3.loc[df_ctrl_3['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
ctrl_discrim_sem_3 = df_ctrl_3.loc[df_ctrl_3['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()


anx_discrim_1 = df_anx_1.loc[df_anx_1['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
anx_discrim_sem_1 = df_anx_1.loc[df_anx_1['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()

anx_discrim_2 = df_anx_2.loc[df_anx_2['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
anx_discrim_sem_2 = df_anx_2.loc[df_anx_2['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()

anx_discrim_3 = df_anx_3.loc[df_anx_3['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
anx_discrim_sem_3 = df_anx_3.loc[df_anx_3['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()


# Create figure
fig = plt.figure(figsize=(18, 8), facecolor='white')
gs = fig.add_gridspec(1, 2)

f_ax1 = fig.add_subplot(gs[0, 0])
plt.errorbar(x = list(ctrl_discrim_1.index), y = ctrl_discrim_1.values, yerr = ctrl_discrim_sem_1.values, color=COLOR_CTRL, label='Ext1')
plt.errorbar(x = list(ctrl_discrim_2.index), y = ctrl_discrim_2.values, yerr = ctrl_discrim_sem_2.values, color=COLOR_CTRL, label='Ext2', alpha=0.7)
plt.errorbar(x = list(ctrl_discrim_3.index), y = ctrl_discrim_3.values, yerr = ctrl_discrim_sem_3.values, color=COLOR_CTRL, label='Ext3', alpha=0.4)


#plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color=COLOR_CTRL, label='control')
plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
plt.title('Discrimination ratios - control group', fontsize=SUBTITLE_SIZE)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel('discrimination ratio', fontsize=AXES_LABEL_SIZE)
plt.legend(loc = 'lower left')

#fig.add_subplot(gs[0,1], sharey=f_ax1)

#plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color='g', label='discrim. ratio')
#plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
#plt.title('Control group')
#plt.xlabel('time in [s] from stimulus onset')
#plt.ylabel('normalized CS responses / discrimination ratio')
#plt.legend(loc = 'lower left')

fig.add_subplot(gs[0,1], sharey=f_ax1)
#fig.add_subplot(gs[0,2], sharey=f_ax1)
plt.errorbar(x = list(anx_discrim_1.index), y = anx_discrim_1.values, yerr = anx_discrim_sem_1.values, color=COLOR_ANX, label='Ext1')
plt.errorbar(x = list(anx_discrim_2.index), y = anx_discrim_2.values, yerr = anx_discrim_sem_2.values, color=COLOR_ANX, label='Ext2', alpha=0.7)
plt.errorbar(x = list(anx_discrim_3.index), y = anx_discrim_3.values, yerr = anx_discrim_sem_3.values, color=COLOR_ANX, label='Ext3', alpha=0.4)

#plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color=COLOR_CTRL, label='control')
plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
plt.title('Discrimination ratios - anxiety group', fontsize=SUBTITLE_SIZE)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel('discrimination ratio', fontsize=AXES_LABEL_SIZE)
plt.legend(loc = 'lower left')

plt.suptitle(MEASUREMENT + ' in session: Ext1-3', fontsize = TITLE_SIZE)
plt.tight_layout()

#plt.savefig('EDA_discrimination_ratios_across_extinction_sessions.png', dpi=300)
plt.show()

## To warp-up the classical "controls vs. anxious" part, let´s have a look at the baseline measures:

In [ ]:
df_individual_subjects = df.loc[(df['measurement'] == 'EDA') & (df['value_type'] == 'abs_CS-') & (df['session'] == 'preacq') & (df['stim_count'] == '1')].copy()

fig = plt.figure(figsize=(18, 8), facecolor='white')
gs = fig.add_gridspec(1, 2)

f_ax1 = fig.add_subplot(gs[0, 0])

dv = 'baseline_HR_Phase1'
group = 'group'

df_normality = pg.normality(df_individual_subjects, dv=dv, group=group)
df_equal_var = pg.homoscedasticity(df_individual_subjects, dv=dv, group=group)

if (df_normality.iloc[0,1] > 0.05) & (df_normality.iloc[1,1] > 0.05) & (df_equal_var.iloc[0,1] > 0.05):
    # parametric test
    x = df_individual_subjects.loc[df_individual_subjects[group] == 'control', dv].values
    y = df_individual_subjects.loc[df_individual_subjects[group] == 'anxiety', dv].values
    df_stats = pg.ttest(x, y)
    p_val = df_stats['p-val']['T-test']
    
else:
    # non-parametric test
    x = df_individual_subjects.loc[df_individual_subjects[group] == 'control', dv].values
    y = df_individual_subjects.loc[df_individual_subjects[group] == 'anxiety', dv].values
    df_stats = pg.mwu(x, y)
    p_val = df_stats['p-val']['MWU']
    
sns.swarmplot(data=df_individual_subjects, x=group, y=dv, ax=f_ax1, palette=['g', 'm'])
plt.xlabel('')
plt.ylabel('heart rate [bpm]')
plt.ylim(0)

if p_val > 0.05:
    title = 'Baseline heart rates are not significantly different (p = {})'.format(str(round(p_val, 4)))  
else:
    title = 'Baseline heart rates are significantly different (p = {})'.format(str(round(p_val, 4)))

plt.title(title)


f_ax2 = fig.add_subplot(gs[0, 1])

dv = 'baseline_RMSSD_Phase1'
group = 'group'

df_normality = pg.normality(df_individual_subjects, dv=dv, group=group)
df_equal_var = pg.homoscedasticity(df_individual_subjects, dv=dv, group=group)

if (df_normality.iloc[0,1] > 0.05) & (df_normality.iloc[1,1] > 0.05) & (df_equal_var.iloc[0,1] > 0.05):
    # parametric test
    x = df_individual_subjects.loc[df_individual_subjects[group] == 'control', dv].values
    y = df_individual_subjects.loc[df_individual_subjects[group] == 'anxiety', dv].values
    df_stats = pg.ttest(x, y)
    p_val = df_stats['p-val']['T-test']
    
else:
    # non-parametric test
    x = df_individual_subjects.loc[df_individual_subjects[group] == 'control', dv].values
    y = df_individual_subjects.loc[df_individual_subjects[group] == 'anxiety', dv].values
    df_stats = pg.mwu(x, y)
    p_val = df_stats['p-val']['MWU']
    
sns.swarmplot(data=df_individual_subjects, x=group, y=dv, ax=f_ax2, palette=['g', 'm'])
plt.xlabel('')
plt.ylabel('RMSSD')
plt.ylim(0)

if p_val > 0.05:
    title = 'Baseline RMSSDs are not significantly different (p = {})'.format(str(round(p_val, 4)))  
else:
    title = 'Baseline RMSSDs are significantly different (p = {})'.format(str(round(p_val, 4)))

plt.title(title)

#plt.savefig('Baseline_measurements_Anx_vs_Ctrl.png', dpi=300)
plt.show()

# Moreover, we also have the Responder & Discriminator classifications

#### In a first go, let´s check for the distribution of responders & discriminator subjects within the two groups

In [ ]:
df_proportions = pd.DataFrame(index=['perc_HR_responder', 'perc_HR_discriminator', 'perc_EDA_responder', 
                                    'perc_EDA_discriminator', 'perc_HR_EDA_responder', 'perc_HR_EDA_discriminator',
                                    'perc_no_responder', 'perc_no_discriminator'],
                             columns=['control', 'anxiety'])

for group in ['control', 'anxiety']:
    all_subjects = len(df.loc[df['group'] == group, 'subject_id'].unique())
    
    perc_HR_responder = (len(df.loc[(df['group'] == group) & (df['HR_responder'] == True) & (df['EDA_responder'] == False), 'subject_id'].unique()) / all_subjects) * 100
    perc_HR_discriminator = (len(df.loc[(df['group'] == group) & (df['HR_discriminator'] == True)  & (df['EDA_discriminator'] == False), 'subject_id'].unique()) / all_subjects) * 100
    
    perc_EDA_responder = (len(df.loc[(df['group'] == group) & (df['EDA_responder'] == True)  & (df['HR_responder'] == False), 'subject_id'].unique()) / all_subjects) * 100
    perc_EDA_discriminator = (len(df.loc[(df['group'] == group) & (df['EDA_discriminator'] == True) & (df['HR_discriminator'] == False), 'subject_id'].unique()) / all_subjects) * 100
    
    perc_HR_and_EDA_responder = (len(df.loc[(df['group'] == group) & (df['EDA_responder'] == True)  & (df['HR_responder'] == True), 'subject_id'].unique()) / all_subjects) * 100
    perc_HR_and_EDA_discriminator = (len(df.loc[(df['group'] == group) & (df['EDA_discriminator'] == True) & (df['HR_discriminator'] == True), 'subject_id'].unique()) / all_subjects) * 100
    
    perc_no_responder = (len(df.loc[(df['group'] == group) & (df['EDA_responder'] == False)  & (df['HR_responder'] == False), 'subject_id'].unique()) / all_subjects) * 100
    perc_no_discriminator = (len(df.loc[(df['group'] == group) & (df['EDA_discriminator'] == False) & (df['HR_discriminator'] == False), 'subject_id'].unique()) / all_subjects) * 100
      
    
    df_proportions[group] = [perc_HR_responder, perc_HR_discriminator, perc_EDA_responder, perc_EDA_discriminator, 
                             perc_HR_and_EDA_responder, perc_HR_and_EDA_discriminator, perc_no_responder, perc_no_discriminator]

df_proportions

#### We can also plot these results and inspect the distributions visually.
Seems like the predominant (only) difference between the control and anxiety group is <br>
that there is a significant proportion of HR_discriminators "missing" in the anxiety group, <br>
which also leads to a larger proportion of subjects that are classified as no discriminator.

In [ ]:
TYPE = 'discriminator'

In [ ]:
dict_for_chi_square = {'responder': {},
                      'discriminator': {}}

for group in ['control', 'anxiety']:

    N_HR_responder = len(df.loc[(df['group'] == group) & (df['HR_responder'] == True) & (df['EDA_responder'] == False), 'subject_id'].unique())
    N_HR_discriminator = len(df.loc[(df['group'] == group) & (df['HR_discriminator'] == True)  & (df['EDA_discriminator'] == False), 'subject_id'].unique())

    N_EDA_responder = len(df.loc[(df['group'] == group) & (df['EDA_responder'] == True)  & (df['HR_responder'] == False), 'subject_id'].unique())
    N_EDA_discriminator = len(df.loc[(df['group'] == group) & (df['EDA_discriminator'] == True) & (df['HR_discriminator'] == False), 'subject_id'].unique())

    N_HR_and_EDA_responder = len(df.loc[(df['group'] == group) & (df['EDA_responder'] == True)  & (df['HR_responder'] == True), 'subject_id'].unique())
    N_HR_and_EDA_discriminator = len(df.loc[(df['group'] == group) & (df['EDA_discriminator'] == True) & (df['HR_discriminator'] == True), 'subject_id'].unique())

    N_no_responder = len(df.loc[(df['group'] == group) & (df['EDA_responder'] == False)  & (df['HR_responder'] == False), 'subject_id'].unique())
    N_no_discriminator = len(df.loc[(df['group'] == group) & (df['EDA_discriminator'] == False) & (df['HR_discriminator'] == False), 'subject_id'].unique())
    
    dict_for_chi_square['responder'][group] = [N_HR_responder, N_EDA_responder, N_HR_and_EDA_responder, N_no_responder]
    dict_for_chi_square['discriminator'][group] = [N_HR_discriminator, N_EDA_discriminator, N_HR_and_EDA_discriminator, N_no_discriminator]

pval = stats.chi2_contingency([dict_for_chi_square[TYPE]['control'], dict_for_chi_square[TYPE]['anxiety']])[1]

if pval > 0.05:
    significance = 'not significantly'
else:
    significance = 'significantly'
    

fig = plt.figure(figsize=(25,15), facecolor='white')
gs = fig.add_gridspec(1, 2)

fig.add_subplot(gs[0,0])

group = 'control'
HR_only = round(df_proportions.loc['perc_HR_' + TYPE, group], 2)
EDA_only = round(df_proportions.loc['perc_EDA_' + TYPE, group], 2)
HR_and_EDA = round(df_proportions.loc['perc_HR_EDA_' + TYPE, group], 2)
No = round(df_proportions.loc['perc_no_' + TYPE, group], 2)

vd = venn3(subsets = (HR_only, EDA_only, HR_and_EDA, No, 0, 0, 0), set_labels = ('HR ' + TYPE, 'EDA ' + TYPE, 'No  ' + TYPE), alpha = 0.75, set_colors=('darkorange', 'dodgerblue', 'red'))

lbl_a = vd.get_label_by_id("A")
x_a, y_a = lbl_a.get_position()
lbl_a.set_position((x_a-0.15, y_a-0.03))

lbl_b = vd.get_label_by_id("B")
x_b, y_b = lbl_b.get_position()
lbl_b.set_position((x_b+0.1, y_a+0.02))

plt.title('Percentage of ' + TYPE + 's among control subjects', fontsize=TITLE_SIZE)


ax2 = fig.add_subplot(gs[0,1])

group = 'anxiety'
HR_only = round(df_proportions.loc['perc_HR_' + TYPE, group], 2)
EDA_only = round(df_proportions.loc['perc_EDA_' + TYPE, group], 2)
HR_and_EDA = round(df_proportions.loc['perc_HR_EDA_' + TYPE, group], 2)
No = round(df_proportions.loc['perc_no_' + TYPE, group], 2)


vd = venn3(subsets = (HR_only, EDA_only, HR_and_EDA, No, 0, 0, 0), set_labels = ('HR ' + TYPE, 'EDA ' + TYPE, 'No  ' + TYPE), alpha = 0.75, set_colors=('darkorange', 'dodgerblue', 'red'))

lbl_a = vd.get_label_by_id("A")
x_a, y_a = lbl_a.get_position()
lbl_a.set_position((x_a-0.15, y_a))

lbl_b = vd.get_label_by_id("B")
x_b, y_b = lbl_b.get_position()
lbl_b.set_position((x_b+0.1, y_a-0.05))

plt.title('Percentage of ' + TYPE + 's among anxiety subjects', fontsize=TITLE_SIZE)



text = 'Proportions are ' + significance + ' different between the two groups (p = ' + str(round(pval, 4)) + ')'

plt.text(x=-2.1, y=-0.7, s=text, fontsize=TITLE_SIZE+3)

#plt.savefig('Percentage_of_discriminators.png', dpi=300)
plt.show()

### Are there actually correlations between being a HR and being a EDA responder (or discriminator)?

Let´s quickly screen all possibilities that exist for significant results (at given significance and power-level thresholds). Following this screening cell, there is a piece of code that let´s you check any combination in more details. Possible "warnings" can arise if the respective group combination occurs less than five times.

In [ ]:
# The screening will only output those combinations where the p-value was lower or equal to PVAL:
PVAL = 0.05

# The screening will only output those combinations where the power was larger than POWER:
POWER = 0.3

In [ ]:
df_individual_subjects = df.loc[(df['measurement'] == 'EDA') & (df['value_type'] == 'abs_CS-') & (df['session'] == 'preacq') & (df['stim_count'] == '1')].iloc[:, :10].copy()

df_responders = df_individual_subjects.loc[(df_individual_subjects['HR_responder'] == True) | (df_individual_subjects['HR_discriminator'] == True) | 
                                                  (df_individual_subjects['EDA_responder'] == True) | (df_individual_subjects['EDA_discriminator'] == True)].copy()

df_non_responders = df_individual_subjects.loc[(df_individual_subjects['HR_responder'] == False) & (df_individual_subjects['HR_discriminator'] == False) & 
                                                  (df_individual_subjects['EDA_responder'] == False) & (df_individual_subjects['EDA_discriminator'] == False)].copy()

l_significant_results = []

for assigned_groups_to_check in ['both', 'control only', 'anxiety only']:
    for responder_types_to_check in ['all', 'responders only', 'non-responders only']:
        for classification_type in ['responder', 'discriminator']:
            
            df_temp = [df_individual_subjects, df_responders, df_non_responders][['all', 'responders only', 'non-responders only'].index(responder_types_to_check)]

            all_chi_resp = pg.chi2_independence(data=df_temp, x='HR_responder', y='EDA_responder', correction=True)
            all_chi_discrim = pg.chi2_independence(data=df_temp, x='HR_discriminator', y='EDA_discriminator', correction=True)

            anx_chi_resp = pg.chi2_independence(data=df_temp.loc[df_temp['group'] == 'anxiety'], x='HR_responder', y='EDA_responder', correction=True)
            anx_chi_discrim = pg.chi2_independence(data=df_temp.loc[df_temp['group'] == 'anxiety'], x='HR_discriminator', y='EDA_discriminator', correction=True)

            ctrl_chi_resp = pg.chi2_independence(data=df_temp.loc[df_temp['group'] == 'control'], x='HR_responder', y='EDA_responder', correction=True)
            ctrl_chi_discrim = pg.chi2_independence(data=df_temp.loc[df_temp['group'] == 'control'], x='HR_discriminator', y='EDA_discriminator', correction=True)

            if classification_type == 'responder':
                results = [all_chi_resp, ctrl_chi_resp, anx_chi_resp][['both', 'control only', 'anxiety only'].index(assigned_groups_to_check)]
            elif classification_type == 'discriminator':
                results = [all_chi_discrim, ctrl_chi_discrim, anx_chi_discrim][['both', 'control only', 'anxiety only'].index(assigned_groups_to_check)]
                
            if (results[2]['pval'][0] <= PVAL) & (results[2]['power'][0] >= POWER):
                l_significant_results.append((assigned_groups_to_check, responder_types_to_check, classification_type))

l_significant_results

In [ ]:
# Please specify, whether the analysis should be performed only for a specific group
# Chose one of: ['both', 'control only', 'anxiety only']
assigned_groups_to_check = 'both'

# Please specify, whether only responders or non-responders should be analyzed, or whether you want to consider all subjects
# Chose one of: ['all', 'responders only', 'non-repsonders only']
responder_types_to_check = 'responders only'

# Finally, please specify, whether you want to check for correelations among responders or discriminators
# Chose one of: ['responder', 'discriminator']
classification_type = 'discriminator'

In [ ]:
df_individual_subjects = df.loc[(df['measurement'] == 'EDA') & (df['value_type'] == 'abs_CS-') & (df['session'] == 'preacq') & (df['stim_count'] == '1')].iloc[:, :10].copy()

df_responders = df_individual_subjects.loc[(df_individual_subjects['HR_responder'] == True) | (df_individual_subjects['HR_discriminator'] == True) | 
                                                  (df_individual_subjects['EDA_responder'] == True) | (df_individual_subjects['EDA_discriminator'] == True)].copy()

df_non_responders = df_individual_subjects.loc[(df_individual_subjects['HR_responder'] == False) & (df_individual_subjects['HR_discriminator'] == False) & 
                                                  (df_individual_subjects['EDA_responder'] == False) & (df_individual_subjects['EDA_discriminator'] == False)].copy()

df_temp = [df_individual_subjects, df_responders, df_non_responders][['all', 'responders only', 'non-responders only'].index(responder_types_to_check)]

all_chi_resp = pg.chi2_independence(data=df_temp, x='HR_responder', y='EDA_responder', correction=True)
all_chi_discrim = pg.chi2_independence(data=df_temp, x='HR_discriminator', y='EDA_discriminator', correction=True)

anx_chi_resp = pg.chi2_independence(data=df_temp.loc[df_temp['group'] == 'anxiety'], x='HR_responder', y='EDA_responder', correction=True)
anx_chi_discrim = pg.chi2_independence(data=df_temp.loc[df_temp['group'] == 'anxiety'], x='HR_discriminator', y='EDA_discriminator', correction=True)

ctrl_chi_resp = pg.chi2_independence(data=df_temp.loc[df_temp['group'] == 'control'], x='HR_responder', y='EDA_responder', correction=True)
ctrl_chi_discrim = pg.chi2_independence(data=df_temp.loc[df_temp['group'] == 'control'], x='HR_discriminator', y='EDA_discriminator', correction=True)

if classification_type == 'responder':
    results = [all_chi_resp, ctrl_chi_resp, anx_chi_resp][['both', 'control only', 'anxiety only'].index(assigned_groups_to_check)]
elif classification_type == 'discriminator':
    results = [all_chi_discrim, ctrl_chi_discrim, anx_chi_discrim][['both', 'control only', 'anxiety only'].index(assigned_groups_to_check)]
    
results

## We can now also compare the responses between different groups of responders / discriminators, ignoring original group assignments (i.e. control / anxiety):

In [ ]:
# Chose either 'single_feature' or 'all_features':
TYPE = 'all_features'

# Specification only required if TYPE == 'single_feature'
FEATURE = 'HR_discriminator'

# Specification only required if TYPE == 'all_features'
GROUP1 = {'HR_responder': True,
          'HR_discriminator': True, 
          'EDA_responder': False,
          'EDA_discriminator': False,
          'color': 'darkorange'}

GROUP2 = {'HR_responder': False,
          'HR_discriminator': False, 
          'EDA_responder': True,
          'EDA_discriminator': True,
          'color': 'dodgerblue'}


### In a first attempt, let´s plot the responses across all sessions to check how consistent the classification is

In [ ]:
MEASUREMENT = 'HR'

In [ ]:
# Depending on whether df is created by reading a .csv file or from preprocessing, the timepoints seem to be stored differently (as int or as str). To catch this error:
if '-1' in df.columns:
    column_a, column_b = '-1', '15'
elif -1 in df.columns:
    column_a, column_b = -1, 15

# To make this plot highly flexible, e.g. if additional columns are added with regard to responder classifications and so on, the column indices are not hard coded:
idx_start = list(df.columns).index(column_a)
idx_stop = list(df.columns).index(column_b)





# Create figure
fig = plt.figure(figsize=(20, 40))
gs = fig.add_gridspec(8, 2)


l_sessions = ['preacq', 'acq1', 'acq2', 'gen1', 'gen2', 'ext1', 'ext2', 'ext3']


SESSION = 'preacq'

# Select data
if TYPE == 'all_features':
    df_grp1 = df.loc[(df['session'] == SESSION) & (df['measurement'] == MEASUREMENT)
                     & (df['HR_responder'] == GROUP1['HR_responder']) & (df['HR_discriminator'] == GROUP1['HR_discriminator'])
                     & (df['EDA_responder'] == GROUP1['EDA_responder']) & (df['EDA_discriminator'] == GROUP1['EDA_discriminator'])].copy()


    df_grp2 = df.loc[(df['session'] == SESSION) & (df['measurement'] == MEASUREMENT)
                     & (df['HR_responder'] == GROUP2['HR_responder']) & (df['HR_discriminator'] == GROUP2['HR_discriminator'])
                     & (df['EDA_responder'] == GROUP2['EDA_responder']) & (df['EDA_discriminator'] == GROUP2['EDA_discriminator'])].copy()

elif TYPE == 'single_feature':
    df_grp1 = df.loc[(df['session'] == SESSION) & (df['measurement'] == MEASUREMENT) & (df[FEATURE] == True)].copy()
    
    df_grp2 = df.loc[(df['session'] == SESSION) & (df['measurement'] == MEASUREMENT) & (df[FEATURE] == False)].copy()

subjects_grp1 = df_grp1['subject_id'].unique().shape[0]
subjects_grp2 = df_grp2['subject_id'].unique().shape[0] 

grp1_norm_cs_p = df_grp1.loc[df_grp1['value_type'] == 'norm_global_CS+'].iloc[:, idx_start:idx_stop].mean()
grp1_norm_cs_m = df_grp1.loc[df_grp1['value_type'] == 'norm_global_CS-'].iloc[:, idx_start:idx_stop].mean()
grp1_discrim = df_grp1.loc[df_grp1['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
grp1_discrim_sem = df_grp1.loc[df_grp1['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()

grp2_norm_cs_p = df_grp2.loc[df_grp2['value_type'] == 'norm_global_CS+'].iloc[:, idx_start:idx_stop].mean()
grp2_norm_cs_m = df_grp2.loc[df_grp2['value_type'] == 'norm_global_CS-'].iloc[:, idx_start:idx_stop].mean()
grp2_discrim = df_grp2.loc[df_grp2['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
grp2_discrim_sem = df_grp2.loc[df_grp2['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()


ax1 = fig.add_subplot(gs[0, 0])
plt.plot(grp1_norm_cs_p, color = GROUP1['color'], alpha = 1, label='Group 1 CS+')
plt.plot(grp1_norm_cs_m, color = GROUP1['color'], alpha = 1, linestyle = 'dashed', label='Group 1 CS-')

plt.plot(grp2_norm_cs_p, color = GROUP2['color'], alpha = 1, label='Group 2 CS+')
plt.plot(grp2_norm_cs_m, color = GROUP2['color'], alpha = 1, linestyle = 'dashed', label='Group 2 CS-')
#plt.errorbar(x = list(anx_discrim.index), y = anx_discrim.values, yerr = anx_discrim_sem.values, color='m', label='discrim. ratio')
#plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel('normalized CS responses', fontsize=AXES_LABEL_SIZE)
plt.title('CS responses during: {} [N_grp1 = {}, N_grp2 = {}]'.format(SESSION, str(subjects_grp1), str(subjects_grp2)), fontsize=SUBTITLE_SIZE)
plt.legend(loc = 'lower left')

#fig.add_subplot(gs[0,1], sharey=f_ax1)

#plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color='g', label='discrim. ratio')
#plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
#plt.title('Control group')
#plt.xlabel('time in [s] from stimulus onset')
#plt.ylabel('normalized CS responses / discrimination ratio')
#plt.legend(loc = 'lower left')

ax2 = fig.add_subplot(gs[0,1])
#fig.add_subplot(gs[0,2], sharey=f_ax1)
plt.errorbar(x = list(grp1_discrim.index), y = grp1_discrim.values, yerr = grp1_discrim_sem.values, color=GROUP1['color'], label='Group 1')
plt.errorbar(x = list(grp2_discrim.index), y = grp2_discrim.values, yerr = grp2_discrim_sem.values, color=GROUP2['color'], label='Group 2')
plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
plt.title('Discrimination ratios during: {} [N_grp1 = {}, N_grp2 = {}]'.format(SESSION, str(subjects_grp1), str(subjects_grp2)), fontsize=SUBTITLE_SIZE)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel('discrimination ratio', fontsize=AXES_LABEL_SIZE)
plt.legend(loc = 'lower left')


for column in range(1,8):
    SESSION = l_sessions[column]

    # Select data
    if TYPE == 'all_features':
        df_grp1 = df.loc[(df['session'] == SESSION) & (df['measurement'] == MEASUREMENT)
                         & (df['HR_responder'] == GROUP1['HR_responder']) & (df['HR_discriminator'] == GROUP1['HR_discriminator'])
                         & (df['EDA_responder'] == GROUP1['EDA_responder']) & (df['EDA_discriminator'] == GROUP1['EDA_discriminator'])].copy()


        df_grp2 = df.loc[(df['session'] == SESSION) & (df['measurement'] == MEASUREMENT)
                         & (df['HR_responder'] == GROUP2['HR_responder']) & (df['HR_discriminator'] == GROUP2['HR_discriminator'])
                         & (df['EDA_responder'] == GROUP2['EDA_responder']) & (df['EDA_discriminator'] == GROUP2['EDA_discriminator'])].copy()

    elif TYPE == 'single_feature':
        df_grp1 = df.loc[(df['session'] == SESSION) & (df['measurement'] == MEASUREMENT) & (df[FEATURE] == True)].copy()

        df_grp2 = df.loc[(df['session'] == SESSION) & (df['measurement'] == MEASUREMENT) & (df[FEATURE] == False)].copy()


    grp1_norm_cs_p = df_grp1.loc[df_grp1['value_type'] == 'norm_global_CS+'].iloc[:, idx_start:idx_stop].mean()
    grp1_norm_cs_m = df_grp1.loc[df_grp1['value_type'] == 'norm_global_CS-'].iloc[:, idx_start:idx_stop].mean()
    grp1_discrim = df_grp1.loc[df_grp1['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
    grp1_discrim_sem = df_grp1.loc[df_grp1['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()

    grp2_norm_cs_p = df_grp2.loc[df_grp2['value_type'] == 'norm_global_CS+'].iloc[:, idx_start:idx_stop].mean()
    grp2_norm_cs_m = df_grp2.loc[df_grp2['value_type'] == 'norm_global_CS-'].iloc[:, idx_start:idx_stop].mean()
    grp2_discrim = df_grp2.loc[df_grp2['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
    grp2_discrim_sem = df_grp2.loc[df_grp2['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()
    
    
    fig.add_subplot(gs[column, 0], sharey=ax1)
    plt.plot(grp1_norm_cs_p, color = GROUP1['color'], alpha = 1, label='Group 1 CS+')
    plt.plot(grp1_norm_cs_m, color = GROUP1['color'], alpha = 1, linestyle = 'dashed', label='Group 1 CS-')

    plt.plot(grp2_norm_cs_p, color = GROUP2['color'], alpha = 1, label='Group 2 CS+')
    plt.plot(grp2_norm_cs_m, color = GROUP2['color'], alpha = 1, linestyle = 'dashed', label='Group 2 CS-')
    #plt.errorbar(x = list(anx_discrim.index), y = anx_discrim.values, yerr = anx_discrim_sem.values, color='m', label='discrim. ratio')
    #plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
    plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
    plt.ylabel('normalized CS responses', fontsize=AXES_LABEL_SIZE)
    plt.title('CS responses during: {}'.format(SESSION), fontsize=SUBTITLE_SIZE)
    plt.legend(loc = 'lower left')

    fig.add_subplot(gs[column,1], sharey=ax2)
    #fig.add_subplot(gs[0,2], sharey=f_ax1)
    plt.errorbar(x = list(grp1_discrim.index), y = grp1_discrim.values, yerr = grp1_discrim_sem.values, color=GROUP1['color'], label='Group 1')
    plt.errorbar(x = list(grp2_discrim.index), y = grp2_discrim.values, yerr = grp2_discrim_sem.values, color=GROUP2['color'], label='Group 2')
    plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
    plt.title('Discrimination ratios during: {}'.format(SESSION), fontsize=SUBTITLE_SIZE)
    plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
    plt.ylabel('discrimination ratio', fontsize=AXES_LABEL_SIZE)
    plt.legend(loc = 'lower left')


plt.tight_layout()
plt.show()


### We can then again look at the statistics for any particular session and measurement: 

In [ ]:
SESSION = 'acq2' 
MEASUREMENT = 'HR'

In [ ]:
# Depending on whether df is created by reading a .csv file or from preprocessing, the timepoints seem to be stored differently (as int or as str). To catch this error:
if '-1' in df.columns:
    column_a, column_b = '-1', '15'
elif -1 in df.columns:
    column_a, column_b = -1, 15

# To make this plot highly flexible, e.g. if additional columns are added with regard to responder classifications and so on, the column indices are not hard coded:
idx_start = list(df.columns).index(column_a)
idx_stop = list(df.columns).index(column_b)


# Select data
if TYPE == 'all_features':
    df_grp1 = df.loc[(df['session'] == SESSION) & (df['measurement'] == MEASUREMENT)
                     & (df['HR_responder'] == GROUP1['HR_responder']) & (df['HR_discriminator'] == GROUP1['HR_discriminator'])
                     & (df['EDA_responder'] == GROUP1['EDA_responder']) & (df['EDA_discriminator'] == GROUP1['EDA_discriminator'])].copy()


    df_grp2 = df.loc[(df['session'] == SESSION) & (df['measurement'] == MEASUREMENT)
                     & (df['HR_responder'] == GROUP2['HR_responder']) & (df['HR_discriminator'] == GROUP2['HR_discriminator'])
                     & (df['EDA_responder'] == GROUP2['EDA_responder']) & (df['EDA_discriminator'] == GROUP2['EDA_discriminator'])].copy()

elif TYPE == 'single_feature':
    df_grp1 = df.loc[(df['session'] == SESSION) & (df['measurement'] == MEASUREMENT) & (df[FEATURE] == True)].copy()
    
    df_grp2 = df.loc[(df['session'] == SESSION) & (df['measurement'] == MEASUREMENT) & (df[FEATURE] == False)].copy()

    
subjects_grp1 = df_grp1['subject_id'].unique().shape[0]
subjects_grp2 = df_grp2['subject_id'].unique().shape[0]    

grp1_norm_cs_p = df_grp1.loc[df_grp1['value_type'] == 'norm_global_CS+'].iloc[:, idx_start:idx_stop].mean()
grp1_norm_cs_m = df_grp1.loc[df_grp1['value_type'] == 'norm_global_CS-'].iloc[:, idx_start:idx_stop].mean()
grp1_discrim = df_grp1.loc[df_grp1['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
grp1_discrim_sem = df_grp1.loc[df_grp1['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()

grp2_norm_cs_p = df_grp2.loc[df_grp2['value_type'] == 'norm_global_CS+'].iloc[:, idx_start:idx_stop].mean()
grp2_norm_cs_m = df_grp2.loc[df_grp2['value_type'] == 'norm_global_CS-'].iloc[:, idx_start:idx_stop].mean()
grp2_discrim = df_grp2.loc[df_grp2['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
grp2_discrim_sem = df_grp2.loc[df_grp2['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()


# Create figure
fig = plt.figure(figsize=(18, 8))
gs = fig.add_gridspec(1, 2)

f_ax1 = fig.add_subplot(gs[0, 0])
plt.plot(grp1_norm_cs_p, color = GROUP1['color'], alpha = 1, label='Group 1 CS+')
plt.plot(grp1_norm_cs_m, color = GROUP1['color'], alpha = 1, linestyle = 'dashed', label='Group 1 CS-')

plt.plot(grp2_norm_cs_p, color = GROUP2['color'], alpha = 1, label='Group 2 CS+')
plt.plot(grp2_norm_cs_m, color = GROUP2['color'], alpha = 1, linestyle = 'dashed', label='Group 2 CS-')
#plt.errorbar(x = list(anx_discrim.index), y = anx_discrim.values, yerr = anx_discrim_sem.values, color='m', label='discrim. ratio')
#plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel('normalized CS responses', fontsize=AXES_LABEL_SIZE)
plt.title('CS responses', fontsize=SUBTITLE_SIZE)
plt.legend(loc = 'lower left')

#fig.add_subplot(gs[0,1], sharey=f_ax1)

#plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color='g', label='discrim. ratio')
#plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
#plt.title('Control group')
#plt.xlabel('time in [s] from stimulus onset')
#plt.ylabel('normalized CS responses / discrimination ratio')
#plt.legend(loc = 'lower left')

fig.add_subplot(gs[0,1])
#fig.add_subplot(gs[0,2], sharey=f_ax1)
plt.errorbar(x = list(grp1_discrim.index), y = grp1_discrim.values, yerr = grp1_discrim_sem.values, color=GROUP1['color'], label='Group 1')
plt.errorbar(x = list(grp2_discrim.index), y = grp2_discrim.values, yerr = grp2_discrim_sem.values, color=GROUP2['color'], label='Group 2')
plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
plt.title('Discrimination ratios', fontsize=SUBTITLE_SIZE)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel('discrimination ratio', fontsize=AXES_LABEL_SIZE)
plt.legend(loc = 'lower left')

plt.suptitle('{} in session: {} [N_grp1 = {}   N_grp2 = {}]'.format(MEASUREMENT, SESSION, str(subjects_grp1), str(subjects_grp2)), fontsize = TITLE_SIZE)
plt.tight_layout()

plt.show()


# Now also compute the corresponding statistics:

VALUE_TYPE = 'discrim_ratio'

# Depending on whether df is created by reading a .csv file or from preprocessing, the timepoints seem to be stored differently (as int or as str). To catch this error:
if '-1' in df.columns:
    l_timepoints = [str(elem) for elem in list(range(0, 16))]
elif -1 in df.columns:
    l_timepoints = list(range(0, 16))

l_dfs_per_timepoint = []

df_grp1['group'] = 'group 1'
df_grp2['group'] = 'group 2'

df_both_groups = pd.concat([df_grp1, df_grp2])

for timepoint in l_timepoints:
    df_temp = df_both_groups.loc[(df_both_groups['session'] == SESSION) & (df_both_groups['value_type'] == 'norm_global_' + VALUE_TYPE) 
                                 & (df_both_groups['measurement'] == MEASUREMENT) & (df_both_groups['stim_count'] == 'session_mean'), 
                                 ['subject_id', 'group', 'measurement', 'session', 'value_type', timepoint]].copy()
    l_headers = list(df_temp.columns)
    l_headers[-1] = 'data'
    df_temp.columns = l_headers
    df_temp['timepoint'] = float(timepoint)
    l_dfs_per_timepoint.append(df_temp)
    
df_stats = pd.concat(l_dfs_per_timepoint)

results_mixed_anova = pg.mixed_anova(data=df_stats, dv='data', between='group', within='timepoint', subject='subject_id')
p_groups, p_timepoints, p_interactions = results_mixed_anova.loc[0,'p-unc'], results_mixed_anova.loc[1,'p-unc'], results_mixed_anova.loc[2,'p-unc']

df_pairwise = pg.pairwise_ttests(data=df_stats, dv='data', between='group', within='timepoint', subject='subject_id', padjust='bonf')
l_pairwise_p_values = list(df_pairwise.loc[df_pairwise['Contrast'] == 'timepoint * group'].sort_values(by='timepoint').loc[:,'p-unc'].values)

p_values_vs_05 = {}
for group in ['group 1', 'group 2']:
    l_p_values_05 = []
    for timepoint in range(0, 16):
        data = df_stats.loc[(df_stats['group'] == group) & (df_stats['timepoint'] == timepoint), 'data'].values
        l_p_values_05.append(round(pg.ttest(x=data, y=0.5).loc['T-test', 'p-val'], 3))
    p_values_vs_05[group] = l_p_values_05

level1 = ['Mixed-Model-ANOVA', 'Mixed-Model-ANOVA', 'Mixed-Model-ANOVA']
level2 = ['Groups', 'Timepoints', 'Interactions']

for x in range(16):
    level1.append('pairwise comparisons per timepoint')
    level2.append(str(x))
    
tuples = list(zip(*[level1, level2]))
index = pd.MultiIndex.from_tuples(tuples)
    
df_stats_summary = pd.DataFrame(index=['p-values Grp1 vs. Grp2', 'p-values Grp1 vs. 0.5', 'p-values Grp2 vs. 0.5'], columns=index)
l_p_values = [p_groups, p_timepoints, p_interactions] + l_pairwise_p_values
l_p_values = [round(elem, 3) for elem in l_p_values]

df_stats_summary.loc['p-values Grp1 vs. Grp2', :] = l_p_values
df_stats_summary.iloc[1, 3:] = p_values_vs_05['group 1']
df_stats_summary.iloc[2, 3:] = p_values_vs_05['group 2']

df_stats_summary

## Similarly, we can also reproduce the generalization and extinction plots with these new group assignments

In [ ]:
GEN_SESSION = 'gen1'
MEASUREMENT = 'HR'

In [ ]:
# Depending on whether df is created by reading a .csv file or from preprocessing, the timepoints seem to be stored differently (as int or as str). To catch this error:
if '-1' in df.columns:
    column_a, column_b = '-1', '15'
elif -1 in df.columns:
    column_a, column_b = -1, 15

# To make this plot highly flexible, e.g. if additional columns are added with regard to responder classifications and so on, the column indices are not hard coded:
idx_start = list(df.columns).index(column_a)
idx_stop = list(df.columns).index(column_b)


# Select data
if TYPE == 'all_features':
    df_grp1 = df.loc[(df['session'] == GEN_SESSION) & (df['measurement'] == MEASUREMENT)
                     & (df['HR_responder'] == GROUP1['HR_responder']) & (df['HR_discriminator'] == GROUP1['HR_discriminator'])
                     & (df['EDA_responder'] == GROUP1['EDA_responder']) & (df['EDA_discriminator'] == GROUP1['EDA_discriminator'])].copy()


    df_grp2 = df.loc[(df['session'] == GEN_SESSION) & (df['measurement'] == MEASUREMENT)
                     & (df['HR_responder'] == GROUP2['HR_responder']) & (df['HR_discriminator'] == GROUP2['HR_discriminator'])
                     & (df['EDA_responder'] == GROUP2['EDA_responder']) & (df['EDA_discriminator'] == GROUP2['EDA_discriminator'])].copy()

elif TYPE == 'single_feature':
    df_grp1 = df.loc[(df['session'] == GEN_SESSION) & (df['measurement'] == MEASUREMENT) & (df[FEATURE] == True)].copy()
    
    df_grp2 = df.loc[(df['session'] == GEN_SESSION) & (df['measurement'] == MEASUREMENT) & (df[FEATURE] == False)].copy()

    
subjects_grp1 = df_grp1['subject_id'].unique().shape[0]
subjects_grp2 = df_grp2['subject_id'].unique().shape[0]

grp1_discrim = df_grp1.loc[df_grp1['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
grp1_discrim_sem = df_grp1.loc[df_grp1['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()

grp1_discrim_GS1 = df_grp1.loc[df_grp1['value_type'] == 'norm_global_discrim_ratio_GS1'].iloc[:, idx_start:idx_stop].mean()
grp1_discrim_GS1_sem = df_grp1.loc[df_grp1['value_type'] == 'norm_global_discrim_ratio_GS1'].iloc[:, idx_start:idx_stop].sem()

grp1_discrim_GS2 = df_grp1.loc[df_grp1['value_type'] == 'norm_global_discrim_ratio_GS2'].iloc[:, idx_start:idx_stop].mean()
grp1_discrim_GS2_sem = df_grp1.loc[df_grp1['value_type'] == 'norm_global_discrim_ratio_GS2'].iloc[:, idx_start:idx_stop].sem()

grp1_discrim_GS3 = df_grp1.loc[df_grp1['value_type'] == 'norm_global_discrim_ratio_GS3'].iloc[:, idx_start:idx_stop].mean()
grp1_discrim_GS3_sem = df_grp1.loc[df_grp1['value_type'] == 'norm_global_discrim_ratio_GS3'].iloc[:, idx_start:idx_stop].sem()

grp1_discrim_GS4 = df_grp1.loc[df_grp1['value_type'] == 'norm_global_discrim_ratio_GS4'].iloc[:, idx_start:idx_stop].mean()
grp1_discrim_GS4_sem = df_grp1.loc[df_grp1['value_type'] == 'norm_global_discrim_ratio_GS4'].iloc[:, idx_start:idx_stop].sem()


grp2_discrim = df_grp2.loc[df_grp2['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
grp2_discrim_sem = df_grp2.loc[df_grp2['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()

grp2_discrim_GS1 = df_grp2.loc[df_grp2['value_type'] == 'norm_global_discrim_ratio_GS1'].iloc[:, idx_start:idx_stop].mean()
grp2_discrim_GS1_sem = df_grp2.loc[df_grp2['value_type'] == 'norm_global_discrim_ratio_GS1'].iloc[:, idx_start:idx_stop].sem()

grp2_discrim_GS2 = df_grp2.loc[df_grp2['value_type'] == 'norm_global_discrim_ratio_GS2'].iloc[:, idx_start:idx_stop].mean()
grp2_discrim_GS2_sem = df_grp2.loc[df_grp2['value_type'] == 'norm_global_discrim_ratio_GS2'].iloc[:, idx_start:idx_stop].sem()

grp2_discrim_GS3 = df_grp2.loc[df_grp2['value_type'] == 'norm_global_discrim_ratio_GS3'].iloc[:, idx_start:idx_stop].mean()
grp2_discrim_GS3_sem = df_grp2.loc[df_grp2['value_type'] == 'norm_global_discrim_ratio_GS3'].iloc[:, idx_start:idx_stop].sem()

grp2_discrim_GS4 = df_grp2.loc[df_grp2['value_type'] == 'norm_global_discrim_ratio_GS4'].iloc[:, idx_start:idx_stop].mean()
grp2_discrim_GS4_sem = df_grp2.loc[df_grp2['value_type'] == 'norm_global_discrim_ratio_GS4'].iloc[:, idx_start:idx_stop].sem()

# Create figure
fig = plt.figure(figsize=(18, 8))
gs = fig.add_gridspec(1, 2)

f_ax1 = fig.add_subplot(gs[0, 0])
plt.errorbar(x = list(grp1_discrim.index), y = grp1_discrim.values, yerr = grp1_discrim_sem.values, color=GROUP1['color'], label='CS-')
plt.errorbar(x = list(grp1_discrim.index), y = grp1_discrim_GS1.values, yerr = grp1_discrim_GS1_sem.values, color=GROUP1['color'], label='GS1', alpha=0.8)
plt.errorbar(x = list(grp1_discrim.index), y = grp1_discrim_GS2.values, yerr = grp1_discrim_GS2_sem.values, color=GROUP1['color'], label='GS2', alpha=0.6)
plt.errorbar(x = list(grp1_discrim.index), y = grp1_discrim_GS3.values, yerr = grp1_discrim_GS3_sem.values, color=GROUP1['color'], label='GS3', alpha=0.4)
plt.errorbar(x = list(grp1_discrim.index), y = grp1_discrim_GS4.values, yerr = grp1_discrim_GS4_sem.values, color=GROUP1['color'], label='GS4', alpha=0.2)

#plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color=COLOR_CTRL, label='control')
plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
plt.title('Discrimination ratios - group 1 [N = {}]'.format(str(subjects_grp1)), fontsize=SUBTITLE_SIZE)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel('discrimination ratio', fontsize=AXES_LABEL_SIZE)
plt.legend(loc = 'lower left')

#fig.add_subplot(gs[0,1], sharey=f_ax1)

#plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color='g', label='discrim. ratio')
#plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
#plt.title('Control group')
#plt.xlabel('time in [s] from stimulus onset')
#plt.ylabel('normalized CS responses / discrimination ratio')
#plt.legend(loc = 'lower left')

fig.add_subplot(gs[0,1], sharey=f_ax1)
#fig.add_subplot(gs[0,2], sharey=f_ax1)
plt.errorbar(x = list(grp2_discrim.index), y = grp2_discrim.values, yerr = grp2_discrim_sem.values, color=GROUP2['color'], label='CS-')
plt.errorbar(x = list(grp2_discrim.index), y = grp2_discrim_GS1.values, yerr = grp2_discrim_GS1_sem.values, color=GROUP2['color'], label='GS1', alpha=0.8)
plt.errorbar(x = list(grp2_discrim.index), y = grp2_discrim_GS2.values, yerr = grp2_discrim_GS2_sem.values, color=GROUP2['color'], label='GS2', alpha=0.6)
plt.errorbar(x = list(grp2_discrim.index), y = grp2_discrim_GS3.values, yerr = grp2_discrim_GS3_sem.values, color=GROUP2['color'], label='GS3', alpha=0.4)
plt.errorbar(x = list(grp2_discrim.index), y = grp2_discrim_GS4.values, yerr = grp2_discrim_GS4_sem.values, color=GROUP2['color'], label='GS4', alpha=0.2)

#plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color=COLOR_CTRL, label='control')
plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
plt.title('Discrimination ratios - group 2 [N = {}]'.format(str(subjects_grp2)), fontsize=SUBTITLE_SIZE)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel('discrimination ratio', fontsize=AXES_LABEL_SIZE)
plt.legend(loc = 'lower left')

plt.suptitle(MEASUREMENT + ' in session: ' + GEN_SESSION, fontsize = TITLE_SIZE)
plt.tight_layout()

plt.show()

In [ ]:
MEASUREMENT = 'HR'

In [ ]:
# Depending on whether df is created by reading a .csv file or from preprocessing, the timepoints seem to be stored differently (as int or as str). To catch this error:
if '-1' in df.columns:
    column_a, column_b = '-1', '15'
elif -1 in df.columns:
    column_a, column_b = -1, 15

# To make this plot highly flexible, e.g. if additional columns are added with regard to responder classifications and so on, the column indices are not hard coded:
idx_start = list(df.columns).index(column_a)
idx_stop = list(df.columns).index(column_b)


# Select data
if TYPE == 'all_features':
    df_grp1_1 = df.loc[(df['session'] == 'ext1') & (df['measurement'] == MEASUREMENT)
                     & (df['HR_responder'] == GROUP1['HR_responder']) & (df['HR_discriminator'] == GROUP1['HR_discriminator'])
                     & (df['EDA_responder'] == GROUP1['EDA_responder']) & (df['EDA_discriminator'] == GROUP1['EDA_discriminator'])].copy()

    df_grp1_2 = df.loc[(df['session'] == 'ext2') & (df['measurement'] == MEASUREMENT)
                     & (df['HR_responder'] == GROUP1['HR_responder']) & (df['HR_discriminator'] == GROUP1['HR_discriminator'])
                     & (df['EDA_responder'] == GROUP1['EDA_responder']) & (df['EDA_discriminator'] == GROUP1['EDA_discriminator'])].copy()

    df_grp1_3 = df.loc[(df['session'] == 'ext3') & (df['measurement'] == MEASUREMENT)
                     & (df['HR_responder'] == GROUP1['HR_responder']) & (df['HR_discriminator'] == GROUP1['HR_discriminator'])
                     & (df['EDA_responder'] == GROUP1['EDA_responder']) & (df['EDA_discriminator'] == GROUP1['EDA_discriminator'])].copy()



    df_grp2_1 = df.loc[(df['session'] == 'ext1') & (df['measurement'] == MEASUREMENT)
                     & (df['HR_responder'] == GROUP2['HR_responder']) & (df['HR_discriminator'] == GROUP2['HR_discriminator'])
                     & (df['EDA_responder'] == GROUP2['EDA_responder']) & (df['EDA_discriminator'] == GROUP2['EDA_discriminator'])].copy()

    df_grp2_2 = df.loc[(df['session'] == 'ext2') & (df['measurement'] == MEASUREMENT)
                     & (df['HR_responder'] == GROUP2['HR_responder']) & (df['HR_discriminator'] == GROUP2['HR_discriminator'])
                     & (df['EDA_responder'] == GROUP2['EDA_responder']) & (df['EDA_discriminator'] == GROUP2['EDA_discriminator'])].copy()

    df_grp2_3 = df.loc[(df['session'] == 'ext3') & (df['measurement'] == MEASUREMENT)
                     & (df['HR_responder'] == GROUP2['HR_responder']) & (df['HR_discriminator'] == GROUP2['HR_discriminator'])
                     & (df['EDA_responder'] == GROUP2['EDA_responder']) & (df['EDA_discriminator'] == GROUP2['EDA_discriminator'])].copy()


elif TYPE == 'single_feature':
    df_grp1_1 = df.loc[(df['session'] == 'ext1') & (df['measurement'] == MEASUREMENT) & (df[FEATURE] == True)].copy()
    df_grp1_2 = df.loc[(df['session'] == 'ext2') & (df['measurement'] == MEASUREMENT) & (df[FEATURE] == True)].copy()
    df_grp1_3 = df.loc[(df['session'] == 'ext3') & (df['measurement'] == MEASUREMENT) & (df[FEATURE] == True)].copy()
    
    df_grp2_1 = df.loc[(df['session'] == 'ext1') & (df['measurement'] == MEASUREMENT) & (df[FEATURE] == False)].copy()
    df_grp2_2 = df.loc[(df['session'] == 'ext2') & (df['measurement'] == MEASUREMENT) & (df[FEATURE] == False)].copy()
    df_grp2_3 = df.loc[(df['session'] == 'ext3') & (df['measurement'] == MEASUREMENT) & (df[FEATURE] == False)].copy()


subjects_grp1 = df_grp1_1['subject_id'].unique().shape[0]
subjects_grp2 = df_grp2_1['subject_id'].unique().shape[0]

grp1_discrim_1 = df_grp1_1.loc[df_grp1_1['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
grp1_discrim_sem_1 = df_grp1_1.loc[df_grp1_1['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()

grp1_discrim_2 = df_grp1_2.loc[df_grp1_2['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
grp1_discrim_sem_2 = df_grp1_2.loc[df_grp1_2['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()

grp1_discrim_3 = df_grp1_3.loc[df_grp1_3['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
grp1_discrim_sem_3 = df_grp1_3.loc[df_grp1_3['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()


grp2_discrim_1 = df_grp2_1.loc[df_grp2_1['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
grp2_discrim_sem_1 = df_grp2_1.loc[df_grp2_1['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()

grp2_discrim_2 = df_grp2_2.loc[df_grp2_2['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
grp2_discrim_sem_2 = df_grp2_2.loc[df_grp2_2['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()

grp2_discrim_3 = df_grp2_3.loc[df_grp2_3['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].mean()
grp2_discrim_sem_3 = df_grp2_3.loc[df_grp2_3['value_type'] == 'norm_global_discrim_ratio'].iloc[:, idx_start:idx_stop].sem()


# Create figure
fig = plt.figure(figsize=(18, 8))
gs = fig.add_gridspec(1, 2)

f_ax1 = fig.add_subplot(gs[0, 0])
plt.errorbar(x = list(grp1_discrim_1.index), y = grp1_discrim_1.values, yerr = grp1_discrim_sem_1.values, color=GROUP1['color'], label='Ext1')
plt.errorbar(x = list(grp1_discrim_2.index), y = grp1_discrim_2.values, yerr = grp1_discrim_sem_2.values, color=GROUP1['color'], label='Ext2', alpha=0.7)
plt.errorbar(x = list(grp1_discrim_3.index), y = grp1_discrim_3.values, yerr = grp1_discrim_sem_3.values, color=GROUP1['color'], label='Ext3', alpha=0.4)


#plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color=COLOR_CTRL, label='control')
plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
plt.title('Discrimination ratios - group 1 [N = {}]'.format(str(subjects_grp1)), fontsize=SUBTITLE_SIZE)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel('discrimination ratio', fontsize=AXES_LABEL_SIZE)
plt.legend(loc = 'lower left')

#fig.add_subplot(gs[0,1], sharey=f_ax1)

#plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color='g', label='discrim. ratio')
#plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
#plt.title('Control group')
#plt.xlabel('time in [s] from stimulus onset')
#plt.ylabel('normalized CS responses / discrimination ratio')
#plt.legend(loc = 'lower left')

fig.add_subplot(gs[0,1], sharey=f_ax1)
#fig.add_subplot(gs[0,2], sharey=f_ax1)
plt.errorbar(x = list(grp2_discrim_1.index), y = grp2_discrim_1.values, yerr = grp2_discrim_sem_1.values, color=GROUP2['color'], label='Ext1')
plt.errorbar(x = list(grp2_discrim_2.index), y = grp2_discrim_2.values, yerr = grp2_discrim_sem_2.values, color=GROUP2['color'], label='Ext2', alpha=0.7)
plt.errorbar(x = list(grp2_discrim_3.index), y = grp2_discrim_3.values, yerr = grp2_discrim_sem_3.values, color=GROUP2['color'], label='Ext3', alpha=0.4)

#plt.errorbar(x = list(ctrl_discrim.index), y = ctrl_discrim.values, yerr = ctrl_disrcim_sem.values, color=COLOR_CTRL, label='control')
plt.hlines(0.5, xmin=-1, xmax=15, color='k', alpha = 0.4)
plt.title('Discrimination ratios - group 2 [N = {}]'.format(str(subjects_grp2)), fontsize=SUBTITLE_SIZE)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel('discrimination ratio', fontsize=AXES_LABEL_SIZE)
plt.legend(loc = 'lower left')

plt.suptitle(MEASUREMENT + ' in session: Ext1-3', fontsize = TITLE_SIZE)
plt.tight_layout()

plt.show()

## We can also check whether there are some correlations between the classifications and baseline measurements:

In [ ]:
df_individual_subjects = df.loc[(df['measurement'] == 'EDA') & (df['value_type'] == 'abs_CS-') & (df['session'] == 'preacq') & (df['stim_count'] == '1')].copy()

df_individual_subjects['temp_group'] = 'remaining'



if TYPE == 'all_features':
    df_individual_subjects.loc[(df_individual_subjects['HR_responder'] == GROUP1['HR_responder']) & (df_individual_subjects['HR_discriminator'] == GROUP1['HR_discriminator']) & 
                               (df_individual_subjects['EDA_responder'] == GROUP1['EDA_responder']) & (df_individual_subjects['EDA_discriminator'] == GROUP1['EDA_discriminator']), 
                               'temp_group'] = 'group1'

    df_individual_subjects.loc[(df_individual_subjects['HR_responder'] == GROUP2['HR_responder']) & (df_individual_subjects['HR_discriminator'] == GROUP2['HR_discriminator']) & 
                               (df_individual_subjects['EDA_responder'] == GROUP2['EDA_responder']) & (df_individual_subjects['EDA_discriminator'] == GROUP2['EDA_discriminator']), 
                               'temp_group'] = 'group2'

elif TYPE == 'single_feature':
    df_individual_subjects.loc[(df_individual_subjects[FEATURE] == True), 'temp_group'] = 'group1'

    df_individual_subjects.loc[(df_individual_subjects[FEATURE] == False), 'temp_group'] = 'group2'


subjects_grp1 = df_individual_subjects.loc[df_individual_subjects['temp_group'] == 'group1', 'subject_id'].unique().shape[0]
subjects_grp2 = df_individual_subjects.loc[df_individual_subjects['temp_group'] == 'group2', 'subject_id'].unique().shape[0]
    

fig = plt.figure(figsize=(18, 8), facecolor='white')
gs = fig.add_gridspec(1, 2)

f_ax1 = fig.add_subplot(gs[0, 0])

dv = 'baseline_HR_Phase1'
group = 'temp_group'

df_normality = pg.normality(df_individual_subjects, dv=dv, group=group)
df_equal_var = pg.homoscedasticity(df_individual_subjects, dv=dv, group=group)

if (df_normality.iloc[0,1] > 0.05) & (df_normality.iloc[1,1] > 0.05) & (df_equal_var.iloc[0,1] > 0.05):
    # parametric test
    x = df_individual_subjects.loc[df_individual_subjects[group] == 'group1', dv].values
    y = df_individual_subjects.loc[df_individual_subjects[group] == 'group2', dv].values
    df_stats = pg.ttest(x, y)
    p_val = df_stats['p-val']['T-test']
    
else:
    # non-parametric test
    x = df_individual_subjects.loc[df_individual_subjects[group] == 'group1', dv].values
    y = df_individual_subjects.loc[df_individual_subjects[group] == 'group2', dv].values
    df_stats = pg.mwu(x, y)
    p_val = df_stats['p-val']['MWU']
    
sns.swarmplot(data=df_individual_subjects, x=group, y=dv, ax=f_ax1, order=['group1', 'group2'], palette=['darkorange', 'dodgerblue'])
plt.xlabel('')
plt.ylabel('heart rate [bpm]')
plt.ylim(0)

if p_val > 0.05:
    title = 'Heart rates are not significantly different (p = {})\n[N_grp1 = {}]            [N_grp2 = {}]'.format(str(round(p_val, 4)), str(subjects_grp1), str(subjects_grp2)) 
else:
    title = 'Heart rates are significantly different (p = {})\n[N_grp1 = {}]            [N_grp2 = {}]'.format(str(round(p_val, 4)), str(subjects_grp1), str(subjects_grp2)) 

plt.title(title)


f_ax2 = fig.add_subplot(gs[0, 1])

dv = 'baseline_RMSSD_Phase1'

df_normality = pg.normality(df_individual_subjects, dv=dv, group=group)
df_equal_var = pg.homoscedasticity(df_individual_subjects, dv=dv, group=group)

if (df_normality.iloc[0,1] > 0.05) & (df_normality.iloc[1,1] > 0.05) & (df_equal_var.iloc[0,1] > 0.05):
    # parametric test
    x = df_individual_subjects.loc[df_individual_subjects[group] == 'group1', dv].values
    y = df_individual_subjects.loc[df_individual_subjects[group] == 'group2', dv].values
    df_stats = pg.ttest(x, y)
    p_val = df_stats['p-val']['T-test']
    
else:
    # non-parametric test
    x = df_individual_subjects.loc[df_individual_subjects[group] == 'group1', dv].values
    y = df_individual_subjects.loc[df_individual_subjects[group] == 'group2', dv].values
    df_stats = pg.mwu(x, y)
    p_val = df_stats['p-val']['MWU']
    
sns.swarmplot(data=df_individual_subjects, x=group, y=dv, ax=f_ax2, order=['group1', 'group2'], palette=['darkorange', 'dodgerblue'])
plt.xlabel('')
plt.ylabel('RMSSD')
plt.ylim(0)

if p_val > 0.05:
    title = 'RMSSDs are not significantly different (p = {})\n[N_grp1 = {}]            [N_grp2 = {}]'.format(str(round(p_val, 4)), str(subjects_grp1), str(subjects_grp2))  
else:
    title = 'RMSSDs are significantly different (p = {})\n[N_grp1 = {}]            [N_grp2 = {}]'.format(str(round(p_val, 4)), str(subjects_grp1), str(subjects_grp2)) 

plt.title(title)

#plt.savefig('Baseline_measurements_HR_discriminators_vs_HR_non_discriminators.png', dpi=300)
plt.show()


### .. some first attempt for an integrated kind of analysis:

In [ ]:
df_int = df.loc[(df['session'] == 'acq2') & (df['value_type'] == 'norm_global_discrim_ratio')].copy()

HUE = df_int.loc[df_int['measurement'] == 'HR', 'group'].values
X = df_int.loc[df_int['measurement'] == 'HR', 5].values
Y = df_int.loc[df_int['measurement'] == 'EDA', 5].values

X_mean_anx = df_int.loc[(df_int['measurement'] == 'HR') & (df_int['group'] == 'anxiety'), 5].mean()
X_mean_ctrl = df_int.loc[(df_int['measurement'] == 'HR') & (df_int['group'] == 'control'), 5].mean()

Y_mean_anx = df_int.loc[(df_int['measurement'] == 'EDA') & (df_int['group'] == 'anxiety'), 5].mean()
Y_mean_ctrl = df_int.loc[(df_int['measurement'] == 'EDA') & (df_int['group'] == 'control'), 5].mean()

plt.figure(figsize=(10,10), facecolor='white')
sns.scatterplot(x=X, y=Y, hue=HUE, palette=['g', 'm'], alpha=0.6)
sns.scatterplot(x=[X_mean_ctrl, X_mean_anx], y=[Y_mean_ctrl, Y_mean_anx] , hue=['control', 'anxiety'], palette=['g', 'm'], sizes=[200], size=[0, 0], legend=False)
plt.xlabel('HR discrimination ratio at 5s during Acq2', fontsize=14)
plt.ylabel('EDA discrimination ratio at 5s during Acq2', fontsize=14)
plt.ylim(0.475, 0.525)
plt.xlim(0.425, 0.575)

plt.show()

## To see all "session mean" responses individually:

In [ ]:
VALUE_TYPE = 'norm_global_CS+'
STIM_COUNT = 'session_mean'
MEASUREMENT = 'HR'
SESSION = 'acq2'

In [ ]:
# Depending on whether df is created by reading a .csv file or from preprocessing, the timepoints seem to be stored differently (as int or as str). To catch this error:
if '-1' in df.columns:
    column_a, column_b = '-1', '15'
elif -1 in df.columns:
    column_a, column_b = -1, 15

# To make this plot highly flexible, e.g. if additional columns are added with regard to responder classifications and so on, the column indices are not hard coded:
idx_start = list(df.columns).index(column_a)
idx_stop = list(df.columns).index(column_b)


df_grp1 = df.loc[(df['session'] == SESSION) & (df['value_type'] == VALUE_TYPE) & (df['measurement'] == MEASUREMENT) & (df['stim_count'] == STIM_COUNT) 
                 & (df['HR_responder'] == GROUP1['HR_responder']) & (df['HR_discriminator'] == GROUP1['HR_discriminator'])
                 & (df['EDA_responder'] == GROUP1['EDA_responder']) & (df['EDA_discriminator'] == GROUP1['EDA_discriminator'])].copy()


df_grp2 = df.loc[(df['session'] == SESSION) & (df['value_type'] == VALUE_TYPE) & (df['measurement'] == MEASUREMENT) & (df['stim_count'] == STIM_COUNT) 
                 & (df['HR_responder'] == GROUP2['HR_responder']) & (df['HR_discriminator'] == GROUP2['HR_discriminator'])
                 & (df['EDA_responder'] == GROUP2['EDA_responder']) & (df['EDA_discriminator'] == GROUP2['EDA_discriminator'])].copy()

mean_grp1 = df_grp1.iloc[:, idx_start:idx_stop].mean()
sem_grp1 = df_grp1.iloc[:, idx_start:idx_stop].sem()
mean_grp2 = df_grp2.iloc[:, idx_start:idx_stop].mean()
sem_grp2 = df_grp2.iloc[:, idx_start:idx_stop].sem()


fig = plt.figure(figsize=(18, 8))
gs = fig.add_gridspec(1, 2)

ax1 = fig.add_subplot(gs[0, 0])
df_grp1.iloc[:, idx_start:idx_stop].transpose().plot(legend=False, alpha=.2, color=GROUP1['color'], ax=ax1)
plt.errorbar(x = list(mean_grp1.index), y = mean_grp1.values, yerr = sem_grp1.values, color=GROUP1['color'])
plt.title('Group 1', fontsize=SUBTITLE_SIZE)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel(VALUE_TYPE, fontsize=AXES_LABEL_SIZE)
#plt.ylim(0.25, 0.4)

ax2 = fig.add_subplot(gs[0, 1], sharey=ax1)
df_grp2.iloc[:, idx_start:idx_stop].transpose().plot(legend=False, alpha=.2, color=GROUP2['color'], ax=ax2)
plt.errorbar(x = list(mean_grp2.index), y = mean_grp2.values, yerr = sem_grp2.values, color=GROUP2['color'])
plt.title('Group 2', fontsize=SUBTITLE_SIZE)
plt.xlabel('time in [s] from stimulus onset', fontsize=AXES_LABEL_SIZE)
plt.ylabel(VALUE_TYPE, fontsize=AXES_LABEL_SIZE)

plt.suptitle(VALUE_TYPE + ' for ' + MEASUREMENT + ' in session: ' + SESSION, fontsize = TITLE_SIZE)
plt.tight_layout()
plt.show()